# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import os
from tqdm import tqdm
import glob
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
from matplotlib import patheffects as pe
from plotly.subplots import make_subplots
import matplotlib
import warnings
import scipy.linalg as linalg
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import pathlib
from sklearn.metrics import mean_absolute_error
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw
import pickle
from src.routines.plotly_layout import add_layout, color_tick
from d3blocks import D3Blocks

# Collect data

In [ ]:
path_load = "D:/YandexDisk/Work/pydnameth/draft/10_MetaEPIClock/MetaEpiAge"
trgt_dirs = ['GPL13534', 'GPL16304', 'GPL21145', 'GPL23976']
paths_gses = []
for trgt_dir in trgt_dirs:
    for path in pathlib.Path(f"{path_load}/{trgt_dir}").rglob("*"):
        if path.is_dir():
            if path.parent in paths_gses:
                paths_gses.remove(path.parent)
            if 'GSE' in str(path):
                paths_gses.append(path)

cols_trgt = [
    'geo_accession',
    'series_id',
    'Age',
    'Sex',
    'Tissue',
    'StateName',
    'Group',
    'Ethnicity',
    'Geography'
]

ages_pc = {
    'PCHorvath1': 'PC-Horvath',
    'PCHorvath2': 'PC-SkinBloodAge',
    'PCHannum': 'PC-Hannum',
    'PCPhenoAge': 'PC-PhenoAge',
    'PCGrimAge': 'PC-GrimAge',
}

pace = 'DunedinPACE'

ages_calc = {
    'DNAmAge': 'Horvath',
    'DNAmAgeHannum': 'Hannum',
    'DNAmPhenoAge': 'PhenoAge',
    'DNAmAgeSkinBloodClock': 'SkinBloodAge',
    'DNAmGrimAgeBasedOnRealAge': 'GrimAge',
    'DNAmGrimAge2BasedOnRealAge': 'GrimAge2',
}

dfs_gses = []

for path_gse in (pbar := tqdm(paths_gses)):
    pbar.set_description(f"Processing {path_gse.parts[-2]}/{path_gse.parts[-1]}")
    
    try:
        df_gse_pheno = pd.read_csv(f"{str(path_gse)}/pheno.csv", index_col=0)
    except UnicodeDecodeError:
        df_gse_pheno = pd.read_csv(f"{str(path_gse)}/pheno.csv", index_col=0, encoding='latin-1')
    except FileNotFoundError:
        print(f"No files for {path_gse.parts[-2]}/{path_gse.parts[-1]}")
        warnings.warn(f"No files for {path_gse.parts[-2]}/{path_gse.parts[-1]}")
        continue
       
    df_gse_pheno['GSE'] = path_gse.parts[-1]
    df_gse_pheno['GPL'] = path_gse.parts[-2]
    df_gse_pheno.rename(columns=ages_pc, inplace=True)
    
    if {'StateName', 'Group', 'Ethnicity', 'Geography'}.issubset(df_gse_pheno.columns): 
        fn_gse_horvath_files = glob.glob(f"{str(path_gse)}/DNAmAgeCalcProject_*_Results.csv")
        if len(fn_gse_horvath_files) > 0:
            fn_gse_horvath = fn_gse_horvath_files[0]
            df_gse_horvath = pd.read_csv(fn_gse_horvath, index_col=0)
            
            df_gse = df_gse_pheno.loc[:, cols_trgt + list(ages_pc.values()) + [pace, 'GSE', 'GPL']]
            for age_col, age_label in ages_calc.items():
                df_gse.loc[df_gse.index.values, age_label] = df_gse_horvath.loc[df_gse.index.values, age_col]
            
            df_gse.set_index('geo_accession', inplace=True)
            dfs_gses.append(df_gse)
        else:
            print(f"No Horvath's for {path_gse.parts[-2]}/{path_gse.parts[-1]}")
    else:
        print(f"Wrong phenotype {path_gse.parts[-2]}/{path_gse.parts[-1]}")

df = pd.concat(dfs_gses, verify_integrity=True)
df.to_excel(f"{path_load}/table.xlsx", index_label='geo_accession')

# Calculate age acceleration

In [ ]:
ref_gse = 'GSE87571'

ages = list(ages_calc.values()) + list(ages_pc.values())
for age_type in (pbar := tqdm(ages)):
    pbar.set_description(f"Processing {age_type}")
    # formula = f"{age_type} ~ Age"
    # model = smf.ols(formula=formula, data=df.loc[df['GSE'] == ref_gse, :]).fit()
    # df[f"{age_type}_linear_pred"] = model.predict(df)
    # df[f"{age_type}Acc"] = df[age_type] - df[f"{age_type}_linear_pred"]
    df[f"{age_type}Acc"] = df[age_type] - df['Age']
    
df.to_excel(f"{path_load}/table.xlsx", index_label='geo_accession')

# TO DELETE: Checking best GSEUNN harmonization

In [ ]:
df_unn = df.loc[df['State'] == 'Russia', :]
gses_unn = df_unn['GSE'].unique()

for harm_type in gses_unn:
    path_save = f"{path_load}/figures/unn_harm_check/{harm_type}"
    pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
    df_tmp = df_unn.loc[df_unn['GSE'] == harm_type, :]
    
    df_stat = pd.DataFrame(index=[f"{x}Acc" for x in ages] + [pace])
    for metric in (pbar := tqdm(df_stat.index.values)):
        pbar.set_description(f"Processing {metric}")
        
        vals = {}
        for group in ['Russians', 'Yakuts']:
            vals[group] = df_tmp.loc[df_tmp['Group'] == group, metric].values
            df_stat.at[metric, f"mean_{group}"] = np.mean(vals[group])
            df_stat.at[metric, f"median_{group}"] = np.median(vals[group])
            df_stat.at[metric, f"q75_{group}"], df_stat.at[metric, f"q25_{group}"] = np.percentile(vals[group], [75 , 25])
            df_stat.at[metric, f"iqr_{group}"] = df_stat.at[metric, f"q75_{group}"] - df_stat.at[metric, f"q25_{group}"]
            
        _, pval = mannwhitneyu(*vals.values(), alternative='two-sided')
        df_stat.at[metric, "pval"] = pval
    
    _, df_stat["pval_fdr_bh"], _, _ = multipletests(df_stat["pval"], 0.05, method='fdr_bh')
    df_stat.to_excel(f"{path_save}/stat.xlsx", index=True)
    
    colors_unn = {'Russians': 'gold', 'Yakuts': 'lightslategray'}
    fig = go.Figure()
    dist_num_bins = 32
    age_order = ages[::-1]
    age_labels = {}
    for age_id, age_type in tqdm(enumerate(age_order)):
        vals_0 = df_tmp.loc[df_tmp['Group'] == 'Russians', f"{age_type}Acc"].values
        color_0 = colors_unn['Russians']
        vals_1 = df_tmp.loc[df_tmp['Group'] == 'Yakuts', f"{age_type}Acc"].values
        color_1 = colors_unn['Yakuts']
        pval = df_stat.at[f'{age_type}Acc', 'pval_fdr_bh']
        age_label = f"{age_type}<br>p-value: {pval:0.2e}"
        age_labels[age_type] = age_label
    
        fig.add_trace(
            go.Violin(
                y=[age_id] * len(vals_0),
                x=vals_0,
                name=age_label,
                box_visible=True,
                meanline_visible=True,
                showlegend=False,
                line_color='black',
                fillcolor=color_0,
                marker=dict(color=color_0, line=dict(color='black', width=0.35), opacity=0.8, size=8),
                points='all',
                bandwidth=np.ptp(vals_0) / dist_num_bins,
                opacity=0.8,
                legendgroup=age_label,
                scalegroup=age_label,
                side='negative',
                orientation='h',
                scalemode="width",
                pointpos=-1.5
            )
        )
    
        fig.add_trace(
            go.Violin(
                y=[age_id] * len(vals_1),
                x=vals_1,
                name=age_label,
                box_visible=True,
                meanline_visible=True,
                showlegend=False,
                line_color='black',
                fillcolor=color_1,
                marker=dict(color=color_1, line=dict(color='black', width=0.35), opacity=0.8, size=8),
                points='all',
                bandwidth=np.ptp(vals_1) / dist_num_bins,
                opacity=0.8,
                legendgroup=age_label,
                scalegroup=age_label,
                scalemode="width",
                side='positive',
                orientation='h',
                pointpos=1.5
            )
        )
    add_layout(fig, "Age acceleration", f"", f"{harm_type}")
    fig.update_layout(
        title=dict(xref='paper', x=0.5),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.01,
            xanchor="left",
            x=0.0001,
            itemsizing='constant',
            font_size=22
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(ages))),
            ticktext=[color_tick('black', age_labels[x]) for x in age_order],
            tickfont=dict(size=25)
        ),
        xaxis=dict(
            tickfont=dict(size=26),
            titlefont=dict(size=26)
        )
    )
    fig.update_layout(
        violingap=0.39,
        violingroupgap=0.39,
        height=140 * len(ages),
        width=1000,
        margin=go.layout.Margin(
            l=260,
            r=30,
            b=110,
            t=50,
            pad=0,
        )
    )
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.update_yaxes(autorange=False, range=[-0.5, len(ages) - 0.5])
    fig.update_xaxes(autorange=True)
    fig.write_image(f"{path_save}/ages_violins.png", scale=2)
    fig.write_image(f"{path_save}/ages_violins.pdf", format="pdf")
    
    fig = go.Figure()
    vals_0 = df_tmp.loc[df_tmp['Group'] == 'Russians', f"DunedinPACE"].values
    color_0 = colors_unn['Russians']
    vals_1 = df_tmp.loc[df_tmp['Group'] == 'Yakuts', f"DunedinPACE"].values
    color_1 = colors_unn['Yakuts']
    pval = df_stat.at[f'DunedinPACE', 'pval_fdr_bh']
    label = f"DunedinPACE<br>p-value: {pval:0.2e}"
    fig.add_trace(
        go.Violin(
            y=[0] * len(vals_0),
            x=vals_0,
            name=label,
            box_visible=True,
            meanline_visible=True,
            showlegend=False,
            line_color='black',
            fillcolor=color_0,
            marker=dict(color=color_0, line=dict(color='black', width=0.35), opacity=0.8, size=8),
            points='all',
            bandwidth=np.ptp(vals_0) / dist_num_bins,
            opacity=0.8,
            legendgroup=label,
            scalegroup=label,
            side='negative',
            orientation='h',
            scalemode="width",
            pointpos=-1.5
        )
    )
    fig.add_trace(
        go.Violin(
            y=[0] * len(vals_1),
            x=vals_1,
            name=label,
            box_visible=True,
            meanline_visible=True,
            showlegend=False,
            line_color='black',
            fillcolor=color_1,
            marker=dict(color=color_1, line=dict(color='black', width=0.35), opacity=0.8, size=8),
            points='all',
            bandwidth=np.ptp(vals_1) / dist_num_bins,
            opacity=0.8,
            legendgroup=label,
            scalegroup=label,
            scalemode="width",
            side='positive',
            orientation='h',
            pointpos=1.5
        )
    )
    add_layout(fig, "DunedinPACE", f"", f"{harm_type}")
    fig.update_layout(
        title=dict(xref='paper', x=0.5),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.01,
            xanchor="left",
            x=0.0001,
            itemsizing='constant',
            font_size=22
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=[0],
            ticktext=[color_tick('black', label)],
            tickfont=dict(size=25)
        ),
        xaxis=dict(
            tickfont=dict(size=26),
            titlefont=dict(size=26)
        )
    )
    fig.update_layout(
        violingap=0.39,
        violingroupgap=0.39,
        height=300,
        width=1000,
        margin=go.layout.Margin(
            l=260,
            r=30,
            b=110,
            t=50,
            pad=0,
        )
    )
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.update_yaxes(autorange=False, range=[-0.5, 0.5])
    fig.update_xaxes(autorange=True)
    fig.write_image(f"{path_save}/DunedinPACE_violins.png", scale=2)
    fig.write_image(f"{path_save}/DunedinPACE_violins.pdf", format="pdf")

# Statistics by state, group, dataset and tissue

## Simplify tissues

In [ ]:
df_tiss = df['Tissue'].value_counts().to_frame()
df_tiss['Tissue Simple'] = df_tiss.index.values
df_tiss.loc[df_tiss.index.str.contains('Blood'), 'Tissue Simple'] = 'Blood'
df_tiss.loc[df_tiss.index.str.contains('Brain'), 'Tissue Simple'] = 'Brain'
df_tiss.loc[df_tiss.index.str.contains('Breast'), 'Tissue Simple'] = 'Breast'
df_tiss.loc[df_tiss.index.str.contains('Fat Adip'), 'Tissue Simple'] = 'Adipose'
tiss_dct_rplc = dict(zip(df_tiss.index, df_tiss['Tissue Simple']))
df.rename(columns={'Tissue': 'Tissue for Calculator'}, inplace=True)
df['Tissue'] = df['Tissue for Calculator']
df['Tissue'].replace(tiss_dct_rplc, inplace=True)
df.to_excel(f"{path_load}/table.xlsx", index_label='geo_accession')

## Exclude groups and states with small number of samples (less than 10)

In [ ]:
df.rename(columns={'StateName': 'State'}, inplace=True)
df_groups = df['Group'].value_counts().to_frame()
groups_to_excl = df_groups.index[df_groups['Group'] < 10].values
df.drop(df[df['Group'].isin(groups_to_excl)].index, inplace=True)
df_states = df['State'].value_counts().to_frame()
states_to_excl = df_states.index[df_states['State'] < 10].values
df.drop(df[df['State'].isin(states_to_excl)].index, inplace=True)
df.to_excel(f"{path_load}/table.xlsx", index_label='geo_accession')

## Get dataframes with counts

In [ ]:
df_tissues_glob = df['Tissue'].value_counts().to_frame()
df_states_glob = df['State'].value_counts().to_frame()
df_groups_glob = df['Group'].value_counts().to_frame()
df_gses_glob = df['GSE'].value_counts().to_frame()
df_tissues_glob.to_excel(f"{path_load}/count_tissues.xlsx", index_label='Tissue')
df_states_glob.to_excel(f"{path_load}/count_states.xlsx", index_label='State')
df_groups_glob.to_excel(f"{path_load}/count_groups.xlsx", index_label='Group')
df_gses_glob.to_excel(f"{path_load}/count_gse.xlsx", index_label='GSE')

tissues = df_tissues_glob.index.values

# Setup colors

In [ ]:
colors_tissues = {
    'Blood': 'red',
    'Buccal': 'skyblue',
    'Brain': 'plum',
    'Colon': 'pink',
    'Epidermis': 'burlywood',
    'Saliva': 'lavender',
    'Lung': 'gold',
    'Muscle': 'darkorange',
    'Breast': 'limegreen',
    'Liver': 'brown',
}

colors_sex = {
    'F': 'red',
    'M': 'blue'
}

colors_xkcd = matplotlib.colors.XKCD_COLORS

df_colors = pd.read_excel(f"{path_load}/colors.xlsx", index_col=0)
colors_global = {}
for feat in df_colors.index.values:
    colors_global[feat] = colors_xkcd[f"xkcd:{df_colors.at[feat, 'xkcd']}"]

# Setup age distribution plots

In [ ]:
ages_kde = {
    'Hannum': {
        'col_id': 0,
        'row_id': 0,
        'title': 'Hannum',
        'y_label': 'Classical clocks',
        'x_label': ''
    },
    'Horvath': {
        'col_id': 1,
        'row_id': 0,
        'title': 'Horvath',
        'y_label': '',
        'x_label': ''
    },
    'SkinBloodAge': {
        'col_id': 2,
        'row_id': 0,
        'title': 'SkinBloodAge',
        'y_label': '',
        'x_label': ''
    },
    'PhenoAge': {
        'col_id': 3,
        'row_id': 0,
        'title': 'PhenoAge',
        'y_label': '',
        'x_label': ''
    },
    'GrimAge': {
        'col_id': 4,
        'row_id': 0,
        'title': 'GrimAge',
        'y_label': '',
        'x_label': ''
    },
    'GrimAge2': {
        'col_id': 5,
        'row_id': 0,
        'title': 'GrimAge2',
        'y_label': '',
        'x_label': 'Age'
    },
    'PC-Hannum': {
        'col_id': 0,
        'row_id': 1,
        'title': '',
        'y_label': 'PC clocks',
        'x_label': 'Age'
    },
    'PC-Horvath': {
        'col_id': 1,
        'row_id': 1,
        'title': '',
        'y_label': '',
        'x_label': 'Age'
    },
    'PC-SkinBloodAge': {
        'col_id': 2,
        'row_id': 1,
        'title': '',
        'y_label': '',
        'x_label': 'Age'
    },
    'PC-PhenoAge': {
        'col_id': 3,
        'row_id': 1,
        'title': '',
        'y_label': '',
        'x_label': 'Age'
    },
    'PC-GrimAge': {
        'col_id': 4,
        'row_id': 1,
        'title': '',
        'y_label': '',
        'x_label': 'Age'
    },
    'PC-GrimAge2': {
        'col_id': 5,
        'row_id': 1,
        'title': '',
        'y_label': '',
        'x_label': ''
    },
}

# Plot tissues

In [ ]:
# Plot sex-specificity
df_ss = pd.DataFrame(index=tissues, columns=['F older M', 'M older F', 'No significance'])
for tissue in tissues:
    df_tissue = df.loc[df['Tissue'] == tissue, :]
    
    path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/sex_specific"
    pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
    
    df_stat_ss = pd.DataFrame(index=ages + [pace])
    for epi_est in ages + [pace]:
        if epi_est == pace:
            col = epi_est
        else:
            col = f"{epi_est}Acc"
        vals_f = df_tissue.loc[df_tissue['Sex'] == 'F',  col].values
        vals_m = df_tissue.loc[df_tissue['Sex'] == 'M',  col].values
        if len(vals_f) > 0 and len(vals_m) > 0:
            _, df_stat_ss.at[epi_est, 'pval'] = mannwhitneyu(vals_f, vals_m, alternative='two-sided')
            if np.mean(vals_f) > np.mean(vals_m):
                df_stat_ss.at[epi_est, 'older'] = 'F'
            else:
                df_stat_ss.at[epi_est, 'older'] = 'M'
        else:
            df_stat_ss.at[epi_est, 'pval'] = 1.0
    _, df_stat_ss['pval_fdr_bh'], _, _ = multipletests(df_stat_ss.loc[:, 'pval'].values, 0.05, method='fdr_bh')
    nzmin = df_stat_ss['pval_fdr_bh'][df_stat_ss['pval_fdr_bh'].gt(0)].min(0) * 0.5
    df_stat_ss['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
    df_stat_ss['pval_fdr_bh_log'] = -np.log10(df_stat_ss.loc[:, 'pval_fdr_bh'].values)
    df_stat_ss.to_excel(f"{path_save}/stat.xlsx")
    
    if df_tissue[df_tissue['Sex'] == 'F'].shape[0] > 0 and df_tissue[df_tissue['Sex'] == 'M'].shape[0] > 0:
        df_ss.at[tissue, 'Insignificant / One sex'] = df_stat_ss[df_stat_ss['pval_fdr_bh'] >= 0.05].shape[0]
        df_ss.at[tissue, 'F older M'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'F')].shape[0]
        df_ss.at[tissue, 'M older F'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'M')].shape[0]
    else:
        df_ss.at[tissue, 'Insignificant / One sex'] = len(ages) + 1
        df_ss.at[tissue, 'F older M'] = 0
        df_ss.at[tissue, 'M older F'] = 0
        
    fig = make_subplots(rows=2, cols=1, row_heights=[11.0, 1.0], vertical_spacing=0.1)
    age_vio_labels = {}
    ages_ordered = [
        'Hannum',
        'PC-Hannum',
        'Horvath',
        'PC-Horvath',
        'SkinBloodAge',
        'PC-SkinBloodAge',
        'PhenoAge',
        'PC-PhenoAge',
        'GrimAge',
        'PC-GrimAge',
        'GrimAge2'
    ][::-1]
    for epi_est_id, epi_est in enumerate(ages_ordered):
        vals_f = df_tissue.loc[df_tissue['Sex'] == 'F',  f"{epi_est}Acc"].values
        vals_m = df_tissue.loc[df_tissue['Sex'] == 'M',  f"{epi_est}Acc"].values
        pval = df_stat_ss.at[epi_est, 'pval_fdr_bh']
        age_vio_labels[epi_est] = f"{epi_est}<br>p-value: {pval:0.2e}"
        if len(vals_f) > 0 and len(vals_m) > 0:
            fig.add_trace(
                go.Violin(
                    y=[epi_est_id] * len(vals_m),
                    x=vals_m,
                    name=age_vio_labels[epi_est],
                    box_visible=True,
                    box_width=0.8,
                    meanline_visible=True,
                    showlegend=False,
                    line_color='black',
                    fillcolor=colors_sex['M'],
                    marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                    points=False,
                    bandwidth=np.ptp(vals_m) / 25,
                    opacity=0.8,
                    legendgroup=age_vio_labels[epi_est],
                    scalegroup=age_vio_labels[epi_est],
                    side='negative',
                    orientation='h',
                    scalemode="width",
                    pointpos=-1.5,
                ),
                row=1,
                col=1
            )
            fig.add_trace(
                go.Violin(
                    y=[epi_est_id] * len(vals_f),
                    x=vals_f,
                    name=age_vio_labels[epi_est],
                    box_visible=True,
                    box_width=0.8,
                    meanline_visible=True,
                    showlegend=False,
                    line_color='black',
                    fillcolor=colors_sex['F'],
                    marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                    points=False,
                    bandwidth=np.ptp(vals_f) / 25,
                    opacity=0.8,
                    legendgroup=age_vio_labels[epi_est],
                    scalegroup=age_vio_labels[epi_est],
                    scalemode="width",
                    side='positive',
                    orientation='h',
                    pointpos=1.5,
                ),
                row=1,
                col=1
            )
    fig.update_layout(template="none")
    fig.update_xaxes(
        title_text="Epigenetic Age Acceleration",
        tickfont=dict(size=26),
        titlefont=dict(size=26),
        showgrid=True,
        autorange=True,
        zeroline=False,
        linecolor='black',
        showline=True,
        gridcolor='gainsboro',
        gridwidth=0.001,
        mirror="allticks",
        ticks='outside',
        showticklabels=True,
        tickangle=0,
        exponentformat='e',
        showexponent='all',
        row=1,
        col=1
    )
    fig.update_yaxes(
        tickmode='array',
        tickvals=list(range(len(ages_ordered))),
        ticktext=[color_tick('black', age_vio_labels[x]) for x in ages_ordered],
        tickfont=dict(size=25),
        showgrid=False,
        autorange=True,
        zeroline=False,
        linecolor='black',
        showline=True,
        gridcolor='gainsboro',
        gridwidth=0.001,
        mirror="allticks",
        ticks='outside',
        showticklabels=True,
        tickangle=0,
        exponentformat='e',
        showexponent='all',
        row=1,
        col=1
    )
    fig.update_yaxes(autorange=False, range=[-0.5, len(ages) - 0.5], row=1, col=1)
    fig.update_xaxes(autorange=True, row=1, col=1)

    vals_f = df_tissue.loc[df_tissue['Sex'] == 'F', pace].values
    vals_m = df_tissue.loc[df_tissue['Sex'] == 'M', pace].values
    pval = df_stat_ss.at[pace, 'pval_fdr_bh']
    age_vio_labels[pace] = f"{pace}<br>p-value: {pval:0.2e}"
    if len(vals_f) > 0 and len(vals_m) > 0:
        fig.add_trace(
            go.Violin(
                y=[0] * len(vals_m),
                x=vals_m,
                name=age_vio_labels[pace],
                box_visible=True,
                box_width=0.8,
                meanline_visible=True,
                showlegend=False,
                line_color='black',
                fillcolor=colors_sex['M'],
                marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                points=False,
                bandwidth=np.ptp(vals_m) / 25,
                opacity=0.8,
                legendgroup=age_vio_labels[pace],
                scalegroup=age_vio_labels[pace],
                side='negative',
                orientation='h',
                scalemode="width",
                pointpos=-1.5,
            ),
            row=2,
            col=1
        )
        fig.add_trace(
            go.Violin(
                y=[0] * len(vals_f),
                x=vals_f,
                name=age_vio_labels[pace],
                box_visible=True,
                box_width=0.8,
                meanline_visible=True,
                showlegend=False,
                line_color='black',
                fillcolor=colors_sex['F'],
                marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                points=False,
                bandwidth=np.ptp(vals_f) / 25,
                opacity=0.8,
                legendgroup=age_vio_labels[pace],
                scalegroup=age_vio_labels[pace],
                scalemode="width",
                side='positive',
                orientation='h',
                pointpos=1.5,
            ),
            row=2,
            col=1
        )
    fig.update_xaxes(
        title_text="Pace of Aging",
        tickfont=dict(size=26),
        titlefont=dict(size=26),
        showgrid=True,
        autorange=True,
        zeroline=False,
        linecolor='black',
        showline=True,
        gridcolor='gainsboro',
        gridwidth=0.001,
        mirror="allticks",
        ticks='outside',
        showticklabels=True,
        tickangle=0,
        exponentformat='e',
        showexponent='all',
        row=2,
        col=1
    )
    fig.update_yaxes(
        tickmode='array',
        tickvals=[0],
        ticktext=[color_tick('black', age_vio_labels[pace])],
        tickfont=dict(size=25),
        showgrid=False,
        autorange=True,
        zeroline=False,
        linecolor='black',
        showline=True,
        gridcolor='gainsboro',
        gridwidth=0.001,
        mirror="allticks",
        ticks='outside',
        showticklabels=True,
        tickangle=0,
        exponentformat='e',
        showexponent='all',
        row=2,
        col=1
    )
    fig.update_yaxes(autorange=False, range=[-0.5, 0.5], row=2, col=1)
    fig.update_xaxes(autorange=True, row=2, col=1)
    
    fig.update_layout(
        title=dict(xref='paper', x=1.0),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.01,
            xanchor="left",
            x=0.0001,
            itemsizing='constant',
            font_size=22
        ),
    )
    fig.update_layout(
        violingap=0.05,
        violingroupgap=0.05,
        height=140 * (len(ages) + 1),
        width=1050,
        margin=go.layout.Margin(
            l=300,
            r=30,
            b=110,
            t=30,
            pad=0,
        )
    )
    fig.write_image(f"{path_save}/violins.png", scale=2)
    fig.write_image(f"{path_save}/violins.pdf", format="pdf")
path_save = f"{path_load}/figures/epi_est_stat/by_tissue"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)    
df_ss.to_excel(f"{path_save}/ss.xlsx", index_label='Tissue')
df_ss['Tissue'] = df_ss.index.values
df_ss_melt = pd.melt(df_ss, id_vars=['Tissue'], value_vars=['F older M', 'M older F', 'Insignificant / One sex'], var_name='EAA type', value_name='Number of EAA estimators')
fig, ax = plt.subplots(figsize=(3, 1.5 + 0.8 * len(tissues)))
sns.set_theme(style='whitegrid')
barplot = sns.barplot(
    data=df_ss_melt,
    y='Tissue',
    hue='EAA type',
    x='Number of EAA estimators',
    edgecolor='black',
    palette={'F older M': 'red', 'M older F': 'blue', 'Insignificant / One sex': 'gray'},
    ax=ax
)
ax.set_xlabel('Number of EAA estimators', fontsize=20)
ax.set_ylabel('', fontsize=20)
ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 23)
ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
for container in ax.containers:
    ax.bar_label(container, fmt='%d', fontsize=13)
sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1))
plt.savefig(f"{path_save}/ss_barplot.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/ss_barplot.pdf", bbox_inches='tight')
plt.close(fig)

n_rows = 2
n_cols = 6
fig_width = 15
fig_height = 8
low_percent = 0.005
hgh_percent = 0.995
ptp_shift = 0.05  
    
for tissue in tissues:
    df_tissue = df.loc[df['Tissue'] == tissue, :]
    
    x_min = df_tissue['Age'].min()
    x_max = df_tissue['Age'].max()
    x_ptp = x_max - x_min
    
    y_min = df_tissue[list(ages)].min().min()
    y_max = df_tissue[list(ages)].max().max()
    y_ptp = y_max - y_min
    
    y_percentiles = df_tissue[ages].quantile([low_percent, hgh_percent])
    y_low = np.min(y_percentiles.loc[low_percent, :].values)
    y_hgh = np.max(y_percentiles.loc[hgh_percent, :].values)
    
    path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/sex_specific"
    pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
    sns.set_theme(style='whitegrid')
    
    for epi_est, params in ages_kde.items():
        if epi_est == 'PC-GrimAge2':
            axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
            axs[params['row_id'], params['col_id']].axis('off')
        else:
            for sex in ['M', 'F']:
                kdeplot = sns.kdeplot(
                    data=df_tissue.loc[df_tissue['Sex'] == sex, :],
                    x='Age',
                    y=epi_est,
                    fill=True,
                    cbar=False,
                    alpha=0.6,
                    cut=0,
                    color=colors_sex[sex],
                    # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                    legend=False,
                    ax=axs[params['row_id'], params['col_id']]
                )
            points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
            axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)
            
            axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
            axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
            axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
            axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
            axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
            if params['x_label'] == '':
                axs[params['row_id'], params['col_id']].set_xticklabels([])
    
    fig.tight_layout()    
    plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
    plt.close(fig)
    
    path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}"
    pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
    
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
    sns.set_theme(style='whitegrid')
    
    for epi_est, params in ages_kde.items():
        if epi_est == 'PC-GrimAge2':
            axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
            axs[params['row_id'], params['col_id']].axis('off')
        else:
            kdeplot = sns.kdeplot(
                data=df_tissue,
                x='Age',
                y=epi_est,
                fill=True,
                cbar=False,
                color=colors_tissues[tissue],
                cut=0,
                # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                legend=False,
                ax=axs[params['row_id'], params['col_id']]
            )
            regplot = sns.regplot(
                data=df_tissue,
                x='Age',
                y=epi_est,
                scatter=False,
                color='black',
                truncate=True,
                ax=axs[params['row_id'], params['col_id']]
            )
            points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
            axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)
            
            corr, _ = stats.pearsonr(df_tissue['Age'].values, df_tissue[epi_est].values)
            mae = mean_absolute_error(df_tissue['Age'].values, df_tissue[epi_est].values)
            label = r'$\rho$ = ' + f"{corr:0.2f}"
            axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.10), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)
            label = f"MAE = {mae:0.2f}"
            axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.02), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)
            
            axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
            axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
            axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
            axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
            axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
            if params['x_label'] == '':
                axs[params['row_id'], params['col_id']].set_xticklabels([])
    
    fig.tight_layout()    
    plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
    plt.close(fig)
    
path_save = f"{path_load}/figures/epi_est_stat/tissues"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

hist_bins = np.linspace(0, 110, 12)
fig, ax = plt.subplots()
sns.set_theme(style='whitegrid')
histplot = sns.histplot(
    data=df,
    x=f"Age",
    bins=hist_bins,
    edgecolor='k',
    linewidth=1,
    hue_order=list(colors_tissues.keys())[::-1],
    hue="Tissue",
    palette=colors_tissues,
    multiple="stack",
    ax=ax
)
plt.savefig(f"{path_save}/histplot.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/histplot.pdf", bbox_inches='tight')
plt.close(fig)

fig, ax = plt.subplots(figsize=(3, 1.5 + 0.8 * len(tissues)))
sns.set_theme(style='whitegrid')
barplot = sns.barplot(
    data=df_tissues_glob,
    y=df_tissues_glob.index.values,
    hue=df_tissues_glob.index.values,
    x=df_tissues_glob['Tissue'].values,
    edgecolor='black',
    palette=colors_tissues,
    dodge=False,
    ax=ax
)
ax.set_xlabel('Number of samples', fontsize=20)
ax.set_ylabel('', fontsize=20)
ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
ax.get_legend().remove()
for container in ax.containers:
    ax.bar_label(container, fmt='%d', fontsize=18)
plt.savefig(f"{path_save}/barplot.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/barplot.pdf", bbox_inches='tight')
plt.close(fig)

df_tissues_aerr_mean = pd.DataFrame(index=tissues, columns=ages, data=np.zeros(shape=(len(tissues), len(ages))))
df_tissues_pace_mean = pd.DataFrame(index=tissues, columns=[pace], data=np.zeros(shape=(len(tissues), 1)))
for tissue in tissues:
    vals = df.loc[df['Tissue'] == tissue, pace].values
    print(f"{tissue}: {len(vals)}")
    df_tissues_pace_mean.at[tissue, pace] = np.mean(vals)
    for age_type in ages:
        vals = df.loc[df['Tissue'] == tissue, f"{age_type}Acc"].values
        df_tissues_aerr_mean.at[tissue, age_type] = np.mean(vals)
df_tissues_aerr_mean.to_excel(f"{path_save}/aerr_mean.xlsx", index_label="Tissue")
df_tissues_pace_mean.to_excel(f"{path_save}/pace_mean.xlsx", index_label="Tissue")

fig, ax = plt.subplots(figsize=(2.4 + 0.37 * len(ages), 1.8 + 0.18 * len(tissues)))
sns.set_theme(style='whitegrid')
heatmap = sns.heatmap(
    df_tissues_aerr_mean,
    annot=True,
    fmt=".1f",
    center=0.0,
    cmap='seismic',
    linewidth=0.1,
    linecolor='black',
    annot_kws={"size": 35 / np.sqrt(max(df_tissues_aerr_mean.shape))},
    ax=ax
)
ax.set_xlabel('Epigenetic Age', fontsize=16)
ax.set_ylabel('Tissues', fontsize=16)
ax.figure.axes[-1].set_ylabel(r"$\langle$ Acceleration $\rangle$", size=16)
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
heatmap_order_ages = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/heatmap_aerr_mean.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/heatmap_aerr_mean.pdf", bbox_inches='tight')
plt.close(fig)

fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(tissues)))
sns.set_theme(style='whitegrid')
violin = sns.violinplot(
    data=df,
    x='Age',
    y='Tissue',
    orient='h',
    inner='box',
    cut=0,
    scale='width',
    order=heatmap_order_ages,
    palette=colors_tissues,
    ax=ax
)
ax.set_xlabel('Chronological Age', fontsize=20)
ax.set_ylabel('Tissues', fontsize=20)
ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 18)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/violin_age_for_heatmap.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/violin_age_for_heatmap.pdf", bbox_inches='tight')
plt.close(fig)

sns.set_theme(style='whitegrid')
clustermap = sns.clustermap(
    df_tissues_aerr_mean,
    annot=True,
    col_cluster=True,
    row_cluster=True,
    fmt=".1f",
    center=0.0,
    cmap='seismic',
    linewidth=0.1,
    linecolor='black',
    tree_kws=dict(linewidths=1.5),
    annot_kws={"size": 55 / np.sqrt(max(df_tissues_aerr_mean.shape))},
    figsize=((0.3 + 0.06 * len(ages)) * 10, (0.37 + 0.04 * len(tissues)) * 10)
)
clustermap.ax_heatmap.set_xlabel('Epigenetic Age', fontsize=20)
clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xmajorticklabels(), fontsize = 18)
clustermap.ax_heatmap.set_ylabel('Tissue', fontsize=20)
clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize = 18)
clustermap.ax_cbar.set_ylabel(r"$\langle$ Acceleration $\rangle$", size=20)
clustermap.ax_cbar.tick_params(labelsize=18)
clustermap_order_ages = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
for spine in clustermap.ax_cbar.spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in clustermap.ax_heatmap.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/clustermap_aerr_mean.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/clustermap_aerr_mean.pdf", bbox_inches='tight')
plt.close(clustermap.fig)

fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(tissues)))
sns.set_theme(style='whitegrid')
violin = sns.violinplot(
    data=df,
    x='Age',
    y='Tissue',
    orient='h',
    inner='box',
    cut=0,
    scale='width',
    order=clustermap_order_ages,
    palette=colors_tissues,
    ax=ax
)
ax.set_xlabel('Chronological Age', fontsize=20)
ax.set_ylabel('Tissues', fontsize=20)
ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 18)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/violin_age_for_clustermap.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/violin_age_for_clustermap.pdf", bbox_inches='tight')
plt.close(fig)

fig, ax = plt.subplots(figsize=(1.5, 1.8 + 0.15 * len(tissues)))
sns.set_theme(style='whitegrid')
heatmap = sns.heatmap(
    df_tissues_pace_mean,
    annot=True,
    fmt=".3f",
    center=1.0,
    cmap='PiYG_r',
    linewidth=0.1,
    linecolor='black',
    annot_kws={"size": 35 / np.sqrt(max(df_tissues_aerr_mean.shape))},
    ax=ax
)
ax.set_xticklabels([''])
ax.set_xlabel('DunedinPACE', fontsize=16)
ax.set_ylabel('Tissues', fontsize=16)
ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$", size=16)
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
heatmap_order_pace = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/heatmap_pace_mean.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/heatmap_pace_mean.pdf", bbox_inches='tight')
plt.close(fig)

fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(tissues)))
sns.set_theme(style='whitegrid')
violin = sns.violinplot(
    data=df,
    x='DunedinPACE',
    y='Tissue',
    orient='h',
    inner='box',
    cut=0,
    scale='width',
    order=heatmap_order_pace,
    palette=colors_tissues,
    ax=ax
)
ax.set_xlabel('DunedinPace', fontsize=20)
ax.set_ylabel('Tissues', fontsize=20)
ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/violin_pace_for_heatmap.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/violin_pace_for_heatmap.pdf", bbox_inches='tight')
plt.close(fig)

sns.set_theme(style='whitegrid')
clustermap = sns.clustermap(
    df_tissues_pace_mean,
    annot=True,
    col_cluster=False,
    row_cluster=True,
    fmt=".3f",
    center=1.0,
    cmap='PiYG_r',
    linewidth=0.1,
    linecolor='black',
    tree_kws=dict(linewidths=1.5),
    dendrogram_ratio=(0.6, 0.0),
    cbar_pos=(0.15, 1.06, 0.9, 0.04),
    cbar_kws={"orientation": "horizontal"},
    annot_kws={"size": 55 / np.sqrt(max(df_tissues_aerr_mean.shape))},
    figsize=(4, (0.25 + 0.03 * len(tissues)) * 10)
)
clustermap.ax_heatmap.set_xlabel('DunedinPACE', fontsize=20)
clustermap.ax_heatmap.set_xticklabels("", fontsize = 18)
clustermap.ax_heatmap.set_ylabel('Tissues', fontsize=20)
clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize = 18)
clustermap.ax_cbar.set_title(r"$\langle$ Pace of Aging $\rangle$", size=20)
clustermap.ax_cbar.tick_params(labelsize=18)
for spine in clustermap.ax_cbar.spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
clustermap_order_pace = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
for tick_label in clustermap.ax_heatmap.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/clustermap_pace_mean.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/clustermap_pace_mean.pdf", bbox_inches='tight')
plt.close(clustermap.fig)

fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(tissues)))
sns.set_theme(style='whitegrid')
violin = sns.violinplot(
    data=df,
    x='DunedinPACE',
    y='Tissue',
    orient='h',
    inner='box',
    cut=0,
    scale='width',
    order=clustermap_order_pace,
    palette=colors_tissues,
    ax=ax
)
ax.set_xlabel('DunedinPace', fontsize=20)
ax.set_ylabel('Tissues', fontsize=20)
ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
plt.savefig(f"{path_save}/violin_pace_for_clustermap.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/violin_pace_for_clustermap.pdf", bbox_inches='tight')
plt.close(fig)

path_save = f"{path_load}/figures/epi_est_stat/tissues/epi_ests"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
dfs_tissues_tissues_stat = {}
for epi_est in ages + [pace]:
    dfs_tissues_tissues_stat[epi_est] = pd.DataFrame(index=tissues, columns=tissues, data=np.zeros(shape=(len(tissues), len(tissues))))
    if epi_est == pace:
        col = epi_est
    else:
        col = f"{epi_est}Acc"
    for tissue_1_id, tissue_1 in enumerate(tissues):
        vals_1 = df.loc[df['Tissue'] == tissue_1,  col].values
        for tissue_2_id in range(tissue_1_id, len(tissues)):
            tissue_2 = tissues[tissue_2_id]
            vals_2 = df.loc[df['Tissue'] == tissue_2, col].values
            if tissue_1 != tissue_2:
                _, pval = mannwhitneyu(vals_1, vals_2, alternative='two-sided')
                diff = np.mean(vals_2) - np.mean(vals_1)
                dfs_tissues_tissues_stat[epi_est].at[tissue_1, tissue_2] = diff
                dfs_tissues_tissues_stat[epi_est].at[tissue_2, tissue_1] = pval
            else:
                dfs_tissues_tissues_stat[epi_est].at[tissue_1, tissue_2]  = np.nan
    selection = np.tri(len(tissues), len(tissues), -1, dtype=np.bool)
    df_fdr = dfs_tissues_tissues_stat[epi_est].where(selection).stack().reset_index()
    df_fdr.columns = ['row', 'col', 'pval']
    _, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
    nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
    df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
    df_fdr['pval_fdr_bh_log'] = -np.log10(df_fdr.loc[:, 'pval_fdr_bh'].values)
    for line_id in range(df_fdr.shape[0]):
        dfs_tissues_tissues_stat[epi_est].loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = df_fdr.at[line_id, 'pval_fdr_bh_log']
    dfs_tissues_tissues_stat[epi_est].to_excel(f"{path_save}/{epi_est}_stat.xlsx", index_label="Tissue")
    
    fig, ax = plt.subplots(figsize=(4.2 + 0.23 * len(tissues), 0.8 + 0.2 * len(tissues)))
    sns.set_theme(style='whitegrid')
    cmap_triu = plt.get_cmap("seismic").copy()
    heatmap_diff = sns.heatmap(
        dfs_tissues_tissues_stat[epi_est],
        mask=np.tri(len(tissues), len(tissues), -1, dtype=np.bool),
        annot=True,
        fmt=".2f",
        center=0.0,
        cmap=cmap_triu,
        linewidth=0.1,
        linecolor='black',
        annot_kws={"size": 25 / np.sqrt(max(df_tissues_aerr_mean.shape))},
        ax=ax
    )
    if epi_est == pace:
        ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$ Difference", size=13)
    else:
        ax.figure.axes[-1].set_ylabel(r"$\langle$ Age Acceleration $\rangle$ Difference", size=13)
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
    cmap_tril = plt.get_cmap("cool").copy()
    cmap_tril.set_under('black')
    heatmap_pval = sns.heatmap(
        dfs_tissues_tissues_stat[epi_est],
        mask=np.tri(len(tissues), len(tissues), -1, dtype=np.bool).T,
        annot=True,
        fmt=".1f",
        vmin=-np.log10(0.05),
        cmap=cmap_tril,
        linewidth=0.1,
        linecolor='black',
        annot_kws={"size": 25 / np.sqrt(max(df_tissues_aerr_mean.shape))},
        ax=ax
    )
    ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$", size=13)
    for spine in ax.figure.axes[-1].spines.values():
        spine.set(visible=True, lw=0.25, edgecolor="black")
    ax.set_xlabel('', fontsize=16)
    ax.set_ylabel('', fontsize=16)
    ax.set_title(epi_est, fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
    for tick_label in ax.get_xticklabels():
        tick_label.set_color(colors_tissues[tick_label.get_text()])
        ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
    for tick_label in ax.get_yticklabels():
        tick_label.set_color(colors_tissues[tick_label.get_text()])
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
    plt.savefig(f"{path_save}/{epi_est}_stat.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path_save}/{epi_est}_stat.pdf", bbox_inches='tight')
    plt.close(fig)
    
df_tissues_tissues_stat = pd.DataFrame(index=tissues, columns=tissues, data=np.zeros(shape=(len(tissues), len(tissues))))
for tissue_1_id, tissue_1 in enumerate(tissues):
    for tissue_2_id in range(tissue_1_id, len(tissues)):
        tissue_2 = tissues[tissue_2_id]
        for epi_est in ages + [pace]:
            pval = dfs_tissues_tissues_stat[epi_est].at[tissue_2, tissue_1]
            diff = dfs_tissues_tissues_stat[epi_est].at[tissue_1, tissue_2]
            if diff > 0:
                df_tissues_tissues_stat.at[tissue_1, tissue_2] += 1
            else:
                df_tissues_tissues_stat.at[tissue_1, tissue_2] -= 1
            df_tissues_tissues_stat.at[tissue_2, tissue_1] += pval / (len(ages + [pace]))
df_tissues_tissues_stat.to_excel(f"{path_save}/global_stat.xlsx", index_label="Tissue")
fig, ax = plt.subplots(figsize=(4.2 + 0.23 * len(tissues), 0.8 + 0.2 * len(tissues)))
sns.set_theme(style='whitegrid')
cmap_triu = plt.get_cmap("seismic").copy()
sns.heatmap(
    df_tissues_tissues_stat,
    mask=np.tri(len(tissues), len(tissues), -1, dtype=np.bool),
    annot=True,
    cmap=cmap_triu,
    center=0.0,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"size": 25 / np.sqrt(max(df_tissues_aerr_mean.shape))},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"Aggregated Sign of $\langle$ EAA $\rangle$ Difference", size=10)
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
cmap_tril = plt.get_cmap("cool").copy()
cmap_tril.set_under('black')
sns.heatmap(
    df_tissues_tissues_stat,
    mask=np.tri(len(tissues), len(tissues), -1, dtype=np.bool).T,
    annot=True,
    fmt=".1f",
    vmin=-np.log10(0.05),
    cmap=cmap_tril,
    linewidth=0.1,
    linecolor='black',
    annot_kws={"size": 25 / np.sqrt(max(df_tissues_aerr_mean.shape))},
    ax=ax
)
ax.figure.axes[-1].set_ylabel(r"$\langle -\log_{10}(\mathrm{p-value}) \rangle$", size=13)
for spine in ax.figure.axes[-1].spines.values():
    spine.set(visible=True, lw=0.25, edgecolor="black")
ax.set_xlabel('', fontsize=16)
ax.set_ylabel('', fontsize=16)
ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in ax.get_xticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
    ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
for tick_label in ax.get_yticklabels():
    tick_label.set_color(colors_tissues[tick_label.get_text()])
    ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
plt.savefig(f"{path_save}/global_stat.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/global_stat.pdf", bbox_inches='tight')
plt.close(fig)
            

# Plot by tissue

## Setup dataframes

In [ ]:
df_states_by_tissue = {}
df_groups_by_tissue = {}
df_gses_by_tissue = {}
dfs_states_group_by_tissue = {}
dfs_states_gse_by_tissue = {}
for tissue in tissues:
    df_tissue = df.loc[df['Tissue'] == tissue, :]
    dfs_states_group_by_tissue[tissue] = {}
    dfs_states_gse_by_tissue[tissue] = {}
    
    df_states_by_tissue[tissue] = df_tissue['State'].value_counts().to_frame()
    df_groups_by_tissue[tissue] = df_tissue['Group'].value_counts().to_frame()
    df_gses_by_tissue[tissue] = df_tissue['GSE'].value_counts().to_frame()
    
    for state in df_states_by_tissue[tissue].index.values:
        dfs_states_group_by_tissue[tissue][state] = df_tissue.loc[df_tissue['State'] == state, :]['Group'].value_counts().to_frame()
        dfs_states_gse_by_tissue[tissue][state] = df_tissue.loc[df_tissue['State'] == state, :]['GSE'].value_counts().to_frame()

## Sankey plot

In [ ]:
sankey_color_dict_by_tissue = {}

for tissue in tissues:
    df_tissue = df.loc[df['Tissue'] == tissue, :]
    path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}"
    pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
    
    sankey_color_dict_by_tissue[tissue] = {}
    sankey_source_dict = {}
    
    sankey_nodes = {
        'pad': 15,
        'thickness': 15,
        'line': {'color': 'black', 'width': 0.5},
        'label': [],
        'color': []
    }
    
    n_nodes = 0
    for gse_id, gse in enumerate(df_gses_by_tissue[tissue].index.values):
        sankey_source_dict[gse] = gse_id + n_nodes
        sankey_color_dict_by_tissue[tissue][gse] = colors_global[gse]
        sankey_nodes['label'].append(gse)
        sankey_nodes['color'].append(sankey_color_dict_by_tissue[tissue][gse])
        
    n_nodes = len(sankey_source_dict)
    for group_id, group in enumerate(df_groups_by_tissue[tissue].index.values):
        sankey_source_dict[group] = group_id + n_nodes
        sankey_color_dict_by_tissue[tissue][group] = colors_global[group]
        sankey_nodes['label'].append(group)
        sankey_nodes['color'].append(sankey_color_dict_by_tissue[tissue][group])
    
    n_nodes = len(sankey_source_dict)
    for state_id, state in enumerate(df_states_by_tissue[tissue].index.values):
        sankey_source_dict[state] = state_id + n_nodes
        sankey_color_dict_by_tissue[tissue][state] = colors_global[state]
        sankey_nodes['label'].append(state)
        sankey_nodes['color'].append(sankey_color_dict_by_tissue[tissue][state])
    
    n_row_elems = max([df_gses_by_tissue[tissue].shape[0], df_groups_by_tissue[tissue].shape[0], df_states_by_tissue[tissue].shape[0]])
    print(f"{tissue}: {n_row_elems}")
    
    sankey_links = {
        'source': [],
        'target': [],
        'value': [],
        'label': [],
        # 'color': [],
        'colorscales': [],
        'line': {'color': 'black', 'width': 0.5},
    }
    sankey_df = pd.DataFrame(columns=['source', 'target', 'weight'])
    for gse in df_gses_by_tissue[tissue].index.values:
        df_link = df_tissue.loc[df_tissue['GSE'] == gse, :]['Group'].value_counts().to_frame()
        for group in df_link.index.values:
            sankey_links['source'].append(sankey_source_dict[gse])
            sankey_links['target'].append(sankey_source_dict[group])
            sankey_links['value'].append(df_link.at[group, 'Group'])
            sankey_links['label'].append(df_link.at[group, 'Group'])
            # sankey_links['color'].append(sankey_color_dict[gse])
            sankey_links['colorscales'].append(dict(colorscale=[[0, sankey_color_dict_by_tissue[tissue][gse]], [1, sankey_color_dict_by_tissue[tissue][group]]]))
            sankey_df = sankey_df.append({'source': gse, 'target': group, 'weight': df_link.at[group, 'Group']}, ignore_index=True)
    
    for group in df_groups_by_tissue[tissue].index.values:
        df_link = df_tissue.loc[df_tissue['Group'] == group, :]['State'].value_counts().to_frame()
        for state in df_link.index.values:
            sankey_links['source'].append(sankey_source_dict[group])
            sankey_links['target'].append(sankey_source_dict[state])
            sankey_links['value'].append(df_link.at[state, 'State'])
            sankey_links['label'].append(df_link.at[state, 'State'])
            # sankey_links['color'].append(sankey_color_dict[group])
            sankey_links['colorscales'].append(dict(colorscale=[[0, sankey_color_dict_by_tissue[tissue][group]], [1, sankey_color_dict_by_tissue[tissue][state]]]))
            sankey_df = sankey_df.append({'source': group, 'target': state, 'weight': df_link.at[state, 'State']}, ignore_index=True)
    
    sankey_df['weight'] = sankey_df['weight'].astype(int)
    
    fig = go.Figure(data=[go.Sankey(
        valueformat="d",
        valuesuffix="",
        node=sankey_nodes,
        link=sankey_links,
    )])
    
    add_layout(fig, "", "", "")
    fig.update_layout(title_xref='paper')
    fig.update_layout(legend_font_size=20)
    fig.update_layout(legend= {'itemsizing': 'constant'})
    fig.update_layout(
        width=600,
        height=100 + n_row_elems * 15,
        margin=go.layout.Margin(
            l=20,
            r=20,
            b=20,
            t=20,
            pad=0,
        )
    )
    fig.update_traces(link_colorscales=sankey_links['colorscales'], selector=dict(type='sankey'))
    fig.write_image(f"{path_save}/sankey.png", scale=2)
    fig.write_image(f"{path_save}/sankey.pdf", format="pdf")
    
    d3 = D3Blocks()
    sankey_color_dict_copy = sankey_color_dict_by_tissue[tissue].copy()
    df_tmp = d3.import_example('energy')
    d3_fig = d3.sankey(sankey_df, figsize=[500, 100 + n_row_elems * 40], color=sankey_color_dict_copy, node={'padding': 5}, fontsize=15)
    d3.show(filepath=f"{path_save}/sankey.html")

## Plot states

In [ ]:
for tissue in tissues:
    print(tissue)
    df_tissue = df.loc[df['Tissue'] == tissue, :]
    
    states = df_states_by_tissue[tissue].index.values
    if len(states) > 1:
        
        colors_states = {state: colors_global[state] for state in states}
        
        # Plot sex-specificity
        df_ss = pd.DataFrame(index=states, columns=['F older M', 'M older F', 'No significance'])
        for state in states:
            df_state = df_tissue.loc[df_tissue['State'] == state,  :]
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/sex_specific"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
            
            df_stat_ss = pd.DataFrame(index=ages + [pace])
            for epi_est in ages + [pace]:
                if epi_est == pace:
                    col = epi_est
                else:
                    col = f"{epi_est}Acc"
                vals_f = df_state.loc[df_state['Sex'] == 'F',  col].values
                vals_m = df_state.loc[df_state['Sex'] == 'M',  col].values
                if len(vals_f) > 0 and len(vals_m) > 0:
                    _, df_stat_ss.at[epi_est, 'pval'] = mannwhitneyu(vals_f, vals_m, alternative='two-sided')
                    if np.mean(vals_f) > np.mean(vals_m):
                        df_stat_ss.at[epi_est, 'older'] = 'F'
                    else:
                        df_stat_ss.at[epi_est, 'older'] = 'M'
                else:
                    df_stat_ss.at[epi_est, 'pval'] = 1.0
            _, df_stat_ss['pval_fdr_bh'], _, _ = multipletests(df_stat_ss.loc[:, 'pval'].values, 0.05, method='fdr_bh')
            nzmin = df_stat_ss['pval_fdr_bh'][df_stat_ss['pval_fdr_bh'].gt(0)].min(0) * 0.5
            df_stat_ss['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
            df_stat_ss['pval_fdr_bh_log'] = -np.log10(df_stat_ss.loc[:, 'pval_fdr_bh'].values)
            df_stat_ss.to_excel(f"{path_save}/stat.xlsx")
            
            if df_state[df_state['Sex'] == 'F'].shape[0] > 0 and df_state[df_state['Sex'] == 'M'].shape[0] > 0:
                df_ss.at[state, 'Insignificant / One sex'] = df_stat_ss[df_stat_ss['pval_fdr_bh'] >= 0.05].shape[0]
                df_ss.at[state, 'F older M'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'F')].shape[0]
                df_ss.at[state, 'M older F'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'M')].shape[0]
            else:
                df_ss.at[state, 'Insignificant / One sex'] = len(ages) + 1
                df_ss.at[state, 'F older M'] = 0
                df_ss.at[state, 'M older F'] = 0
                
            fig = make_subplots(rows=2, cols=1, row_heights=[11.0, 1.0], vertical_spacing=0.1)
            age_vio_labels = {}
            ages_ordered = [
                'Hannum',
                'PC-Hannum',
                'Horvath',
                'PC-Horvath',
                'SkinBloodAge',
                'PC-SkinBloodAge',
                'PhenoAge',
                'PC-PhenoAge',
                'GrimAge',
                'PC-GrimAge',
                'GrimAge2'
            ][::-1]
            for epi_est_id, epi_est in enumerate(ages_ordered):
                vals_f = df_state.loc[df_state['Sex'] == 'F',  f"{epi_est}Acc"].values
                vals_m = df_state.loc[df_state['Sex'] == 'M',  f"{epi_est}Acc"].values
                pval = df_stat_ss.at[epi_est, 'pval_fdr_bh']
                age_vio_labels[epi_est] = f"{epi_est}<br>p-value: {pval:0.2e}"
                if len(vals_f) > 0 and len(vals_m) > 0:
                    fig.add_trace(
                        go.Violin(
                            y=[epi_est_id] * len(vals_m),
                            x=vals_m,
                            name=age_vio_labels[epi_est],
                            box_visible=True,
                            box_width=0.8,
                            meanline_visible=True,
                            showlegend=False,
                            line_color='black',
                            fillcolor=colors_sex['M'],
                            marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                            points=False,
                            bandwidth=np.ptp(vals_m) / 25,
                            opacity=0.8,
                            legendgroup=age_vio_labels[epi_est],
                            scalegroup=age_vio_labels[epi_est],
                            side='negative',
                            orientation='h',
                            scalemode="width",
                            pointpos=-1.5,
                        ),
                        row=1,
                        col=1
                    )
                    fig.add_trace(
                        go.Violin(
                            y=[epi_est_id] * len(vals_f),
                            x=vals_f,
                            name=age_vio_labels[epi_est],
                            box_visible=True,
                            box_width=0.8,
                            meanline_visible=True,
                            showlegend=False,
                            line_color='black',
                            fillcolor=colors_sex['F'],
                            marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                            points=False,
                            bandwidth=np.ptp(vals_f) / 25,
                            opacity=0.8,
                            legendgroup=age_vio_labels[epi_est],
                            scalegroup=age_vio_labels[epi_est],
                            scalemode="width",
                            side='positive',
                            orientation='h',
                            pointpos=1.5,
                        ),
                        row=1,
                        col=1
                    )
            fig.update_layout(template="none")
            fig.update_xaxes(
                title_text="Epigenetic Age Acceleration",
                tickfont=dict(size=26),
                titlefont=dict(size=26),
                showgrid=True,
                autorange=True,
                zeroline=False,
                linecolor='black',
                showline=True,
                gridcolor='gainsboro',
                gridwidth=0.001,
                mirror="allticks",
                ticks='outside',
                showticklabels=True,
                tickangle=0,
                exponentformat='e',
                showexponent='all',
                row=1,
                col=1
            )
            fig.update_yaxes(
                tickmode='array',
                tickvals=list(range(len(ages_ordered))),
                ticktext=[color_tick('black', age_vio_labels[x]) for x in ages_ordered],
                tickfont=dict(size=25),
                showgrid=False,
                autorange=True,
                zeroline=False,
                linecolor='black',
                showline=True,
                gridcolor='gainsboro',
                gridwidth=0.001,
                mirror="allticks",
                ticks='outside',
                showticklabels=True,
                tickangle=0,
                exponentformat='e',
                showexponent='all',
                row=1,
                col=1
            )
            fig.update_yaxes(autorange=False, range=[-0.5, len(ages) - 0.5], row=1, col=1)
            fig.update_xaxes(autorange=True, row=1, col=1)
        
            vals_f = df_state.loc[df_state['Sex'] == 'F', pace].values
            vals_m = df_state.loc[df_state['Sex'] == 'M', pace].values
            pval = df_stat_ss.at[pace, 'pval_fdr_bh']
            age_vio_labels[pace] = f"{pace}<br>p-value: {pval:0.2e}"
            if len(vals_f) > 0 and len(vals_m) > 0:
                fig.add_trace(
                    go.Violin(
                        y=[0] * len(vals_m),
                        x=vals_m,
                        name=age_vio_labels[pace],
                        box_visible=True,
                        box_width=0.8,
                        meanline_visible=True,
                        showlegend=False,
                        line_color='black',
                        fillcolor=colors_sex['M'],
                        marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                        points=False,
                        bandwidth=np.ptp(vals_m) / 25,
                        opacity=0.8,
                        legendgroup=age_vio_labels[pace],
                        scalegroup=age_vio_labels[pace],
                        side='negative',
                        orientation='h',
                        scalemode="width",
                        pointpos=-1.5,
                    ),
                    row=2,
                    col=1
                )
                fig.add_trace(
                    go.Violin(
                        y=[0] * len(vals_f),
                        x=vals_f,
                        name=age_vio_labels[pace],
                        box_visible=True,
                        box_width=0.8,
                        meanline_visible=True,
                        showlegend=False,
                        line_color='black',
                        fillcolor=colors_sex['F'],
                        marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                        points=False,
                        bandwidth=np.ptp(vals_f) / 25,
                        opacity=0.8,
                        legendgroup=age_vio_labels[pace],
                        scalegroup=age_vio_labels[pace],
                        scalemode="width",
                        side='positive',
                        orientation='h',
                        pointpos=1.5,
                    ),
                    row=2,
                    col=1
                )
            fig.update_xaxes(
                title_text="Pace of Aging",
                tickfont=dict(size=26),
                titlefont=dict(size=26),
                showgrid=True,
                autorange=True,
                zeroline=False,
                linecolor='black',
                showline=True,
                gridcolor='gainsboro',
                gridwidth=0.001,
                mirror="allticks",
                ticks='outside',
                showticklabels=True,
                tickangle=0,
                exponentformat='e',
                showexponent='all',
                row=2,
                col=1
            )
            fig.update_yaxes(
                tickmode='array',
                tickvals=[0],
                ticktext=[color_tick('black', age_vio_labels[pace])],
                tickfont=dict(size=25),
                showgrid=False,
                autorange=True,
                zeroline=False,
                linecolor='black',
                showline=True,
                gridcolor='gainsboro',
                gridwidth=0.001,
                mirror="allticks",
                ticks='outside',
                showticklabels=True,
                tickangle=0,
                exponentformat='e',
                showexponent='all',
                row=2,
                col=1
            )
            fig.update_yaxes(autorange=False, range=[-0.5, 0.5], row=2, col=1)
            fig.update_xaxes(autorange=True, row=2, col=1)
            
            fig.update_layout(
                title=dict(xref='paper', x=1.0),
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.01,
                    xanchor="left",
                    x=0.0001,
                    itemsizing='constant',
                    font_size=22
                ),
            )
            fig.update_layout(
                violingap=0.05,
                violingroupgap=0.05,
                height=140 * (len(ages) + 1),
                width=1050,
                margin=go.layout.Margin(
                    l=300,
                    r=30,
                    b=110,
                    t=30,
                    pad=0,
                )
            )
            fig.write_image(f"{path_save}/violins.png", scale=2)
            fig.write_image(f"{path_save}/violins.pdf", format="pdf")
        path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states"
        pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)    
        df_ss.to_excel(f"{path_save}/ss.xlsx", index_label='State')
        df_ss['State'] = df_ss.index.values
        df_ss_melt = pd.melt(df_ss, id_vars=['State'], value_vars=['F older M', 'M older F', 'Insignificant / One sex'], var_name='EAA type', value_name='Number of EAA estimators')
        fig, ax = plt.subplots(figsize=(3, 1.5 + 0.8 * len(states)))
        sns.set_theme(style='whitegrid')
        barplot = sns.barplot(
            data=df_ss_melt,
            y='State',
            hue='EAA type',
            x='Number of EAA estimators',
            edgecolor='black',
            palette={'F older M': 'red', 'M older F': 'blue', 'Insignificant / One sex': 'gray'},
            ax=ax
        )
        ax.set_xlabel('Number of EAA estimators', fontsize=20)
        ax.set_ylabel('', fontsize=20)
        ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize = 23)
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        for container in ax.containers:
            ax.bar_label(container, fmt='%d', fontsize=13)
        sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1))
        plt.savefig(f"{path_save}/ss_barplot.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/ss_barplot.pdf", bbox_inches='tight')
        plt.close(fig)
        
        n_rows = 2
        n_cols = 6
        fig_width = 15
        fig_height = 8
        low_percent = 0.005
        hgh_percent = 0.995
        ptp_shift = 0.1

        for state in states:
            df_state = df_tissue.loc[df_tissue['State'] == state,  :]

            x_min = df_state['Age'].min()
            x_max = df_state['Age'].max()
            x_ptp = x_max - x_min

            y_min = df_state[list(ages)].min().min()
            y_max = df_state[list(ages)].max().max()
            y_ptp = y_max - y_min

            y_percentiles = df_state[ages].quantile([low_percent, hgh_percent])
            y_low = np.min(y_percentiles.loc[low_percent, :].values)
            y_hgh = np.max(y_percentiles.loc[hgh_percent, :].values)
            
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/sex_specific"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
            
            fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
            sns.set_theme(style='whitegrid')
            
            for epi_est, params in ages_kde.items():
                if epi_est == 'PC-GrimAge2':
                    axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                    axs[params['row_id'], params['col_id']].axis('off')
                else:
                    for sex in ['M', 'F']:
                        kdeplot = sns.kdeplot(
                            data=df_state.loc[df_state['Sex'] == sex, :],
                            x='Age',
                            y=epi_est,
                            fill=True,
                            cbar=False,
                            alpha=0.6,
                            cut=0,
                            color=colors_sex[sex],
                            # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                            legend=False,
                            ax=axs[params['row_id'], params['col_id']]
                        )
                    points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
                    axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)
                    
                    axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
                    axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
                    axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
                    axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
                    axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                    if params['x_label'] == '':
                        axs[params['row_id'], params['col_id']].set_xticklabels([])
            
            fig.tight_layout()    
            plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
            plt.close(fig)
            
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
            
            fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
            sns.set_theme(style='whitegrid')

            for epi_est, params in ages_kde.items():
                if epi_est == 'PC-GrimAge2':
                    axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                    axs[params['row_id'], params['col_id']].axis('off')
                else:
                    kdeplot = sns.kdeplot(
                        data=df_state,
                        x='Age',
                        y=epi_est,
                        fill=True,
                        cbar=False,
                        color=colors_states[state],
                        cut=0,
                        # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                        legend=False,
                        ax=axs[params['row_id'], params['col_id']]
                    )
                    regplot = sns.regplot(
                        data=df_state,
                        x='Age',
                        y=epi_est,
                        scatter=False,
                        color='black',
                        truncate=True,
                        ax=axs[params['row_id'], params['col_id']]
                    )
                    points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
                    axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)

                    corr, _ = stats.pearsonr(df_state['Age'].values, df_state[epi_est].values)
                    mae = mean_absolute_error(df_state['Age'].values, df_state[epi_est].values)
                    label = r'$\rho$ = ' + f"{corr:0.2f}"
                    axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.10), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)
                    label = f"MAE = {mae:0.2f}"
                    axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.02), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)

                    axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
                    axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
                    axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
                    axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
                    axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                    if params['x_label'] == '':
                        axs[params['row_id'], params['col_id']].set_xticklabels([])

            fig.tight_layout()    
            plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
            plt.close(fig)
        
        path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/states"
        pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
        
        hist_bins = np.linspace(0, 110, 12)
        fig, ax = plt.subplots()
        sns.set_theme(style='whitegrid')
        histplot = sns.histplot(
            data=df_tissue,
            x=f"Age",
            bins=hist_bins,
            edgecolor='k',
            linewidth=1,
            hue_order=list(colors_states.keys())[::-1],
            hue="State",
            palette=colors_states,
            multiple="stack",
            ax=ax
        )
        plt.savefig(f"{path_save}/histplot.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/histplot.pdf", bbox_inches='tight')
        plt.close(fig)
        
        fig, ax = plt.subplots(figsize=(3, 1.5 + 0.8 * len(states)))
        sns.set_theme(style='whitegrid')
        barplot = sns.barplot(
            data=df_states_by_tissue[tissue],
            y=df_states_by_tissue[tissue].index.values,
            hue=df_states_by_tissue[tissue].index.values,
            x=df_states_by_tissue[tissue]['State'].values,
            edgecolor='black',
            palette=colors_states,
            dodge=False,
            ax=ax
        )
        ax.set_xlabel('Number of samples', fontsize=20)
        ax.set_ylabel('', fontsize=20)
        ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
        ax.get_legend().remove()
        for container in ax.containers:
            ax.bar_label(container, fmt='%d', fontsize=18)
        plt.savefig(f"{path_save}/barplot.png", bbox_inches='tight', dpi=400)
        plt.savefig(f"{path_save}/barplot.pdf", bbox_inches='tight')
        plt.close(fig)
                
        df_states_aerr_mean = pd.DataFrame(index=states, columns=ages, data=np.zeros(shape=(len(states), len(ages))))
        df_states_pace_mean = pd.DataFrame(index=states, columns=[pace], data=np.zeros(shape=(len(states), 1)))
        for state in states:
            vals = df_tissue.loc[df_tissue['State'] == state, pace].values
            df_states_pace_mean.at[state, pace] = np.mean(vals)
            for age_type in ages:
                vals = df_tissue.loc[df_tissue['State'] == state, f"{age_type}Acc"].values
                df_states_aerr_mean.at[state, age_type] = np.mean(vals)
        df_states_aerr_mean.to_excel(f"{path_save}/aerr_mean.xlsx", index_label="State")
        df_states_pace_mean.to_excel(f"{path_save}/pace_mean.xlsx", index_label="State")
        
        fig, ax = plt.subplots(figsize=(2.4 + 0.3 * len(ages), 1.8 + 0.175 * len(states)))
        sns.set_theme(style='whitegrid')
        heatmap = sns.heatmap(
            df_states_aerr_mean,
            annot=True,
            fmt=".1f",
            center=0.0,
            cmap='seismic',
            linewidth=0.1,
            linecolor='black',
            annot_kws={"size": 35 / np.sqrt(max(df_states_aerr_mean.shape))},
            ax=ax
        )
        ax.set_xlabel('Epigenetic Age', fontsize=16)
        ax.set_ylabel('Countries', fontsize=16)
        ax.figure.axes[-1].set_ylabel(r"$\langle$ Acceleration $\rangle$", size=16)
        for spine in ax.figure.axes[-1].spines.values():
            spine.set(visible=True, lw=0.25, edgecolor="black")
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        heatmap_order_ages = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/heatmap_aerr_mean.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/heatmap_aerr_mean.pdf", bbox_inches='tight')
        plt.close(fig)
        
        fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(states)))
        sns.set_theme(style='whitegrid')
        violin = sns.violinplot(
            data=df_tissue,
            x='Age',
            y='State',
            orient='h',
            inner='box',
            cut=0,
            scale='width',
            order=heatmap_order_ages,
            palette=colors_states,
            ax=ax
        )
        ax.set_xlabel('Chronological Age', fontsize=20)
        ax.set_ylabel('Countries', fontsize=20)
        ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 20)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize = 23)
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/violin_age_for_heatmap.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/violin_age_for_heatmap.pdf", bbox_inches='tight')
        plt.close(fig)
        
        sns.set_theme(style='whitegrid')
        clustermap = sns.clustermap(
            df_states_aerr_mean,
            annot=True,
            col_cluster=True,
            row_cluster=True,
            fmt=".1f",
            center=0.0,
            cmap='seismic',
            linewidth=0.1,
            linecolor='black',
            tree_kws=dict(linewidths=1.5),
            annot_kws={"size": 55 / np.sqrt(max(df_states_aerr_mean.shape))},
            figsize=((0.3 + 0.05 * len(ages)) * 10, (0.35 + 0.035 * len(states)) * 10)
        )
        clustermap.ax_heatmap.set_xlabel('Epigenetic Age', fontsize=20)
        clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xmajorticklabels(), fontsize = 18)
        clustermap.ax_heatmap.set_ylabel('Countries', fontsize=20)
        clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize = 18)
        clustermap.ax_cbar.set_ylabel(r"$\langle$ Acceleration $\rangle$", size=20)
        clustermap.ax_cbar.tick_params(labelsize=18)
        clustermap_order_ages = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
        for spine in clustermap.ax_cbar.spines.values():
            spine.set(visible=True, lw=0.25, edgecolor="black")
        clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in clustermap.ax_heatmap.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/clustermap_aerr_mean.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/clustermap_aerr_mean.pdf", bbox_inches='tight')
        plt.close(clustermap.fig)
        
        fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(states)))
        sns.set_theme(style='whitegrid')
        violin = sns.violinplot(
            data=df_tissue,
            x='Age',
            y='State',
            orient='h',
            inner='box',
            cut=0,
            scale='width',
            order=clustermap_order_ages,
            palette=colors_states,
            ax=ax
        )
        ax.set_xlabel('Chronological Age', fontsize=20)
        ax.set_ylabel('Countries', fontsize=20)
        ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 20)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize = 23)
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/violin_age_for_clustermap.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/violin_age_for_clustermap.pdf", bbox_inches='tight')
        plt.close(fig)
        
        fig, ax = plt.subplots(figsize=(1.5, 1.8 + 0.15 * len(states)))
        sns.set_theme(style='whitegrid')
        heatmap = sns.heatmap(
            df_states_pace_mean,
            annot=True,
            fmt=".3f",
            center=1.0,
            cmap='PiYG_r',
            linewidth=0.1,
            linecolor='black',
            annot_kws={"size": 35 / np.sqrt(max(df_states_aerr_mean.shape))},
            ax=ax
        )
        ax.set_xticklabels([''])
        ax.set_xlabel('DunedinPACE', fontsize=16)
        ax.set_ylabel('Countries', fontsize=16)
        ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$", size=16)
        for spine in ax.figure.axes[-1].spines.values():
            spine.set(visible=True, lw=0.25, edgecolor="black")
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        heatmap_order_pace = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/heatmap_pace_mean.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/heatmap_pace_mean.pdf", bbox_inches='tight')
        plt.close(fig)
        
        fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(states)))
        sns.set_theme(style='whitegrid')
        violin = sns.violinplot(
            data=df_tissue,
            x='DunedinPACE',
            y='State',
            orient='h',
            inner='box',
            cut=0,
            scale='width',
            order=heatmap_order_pace,
            palette=colors_states,
            ax=ax
        )
        ax.set_xlabel('DunedinPace', fontsize=20)
        ax.set_ylabel('Countries', fontsize=20)
        ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize = 23)
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/violin_pace_for_heatmap.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/violin_pace_for_heatmap.pdf", bbox_inches='tight')
        plt.close(fig)
        
        sns.set_theme(style='whitegrid')
        clustermap = sns.clustermap(
            df_states_pace_mean,
            annot=True,
            col_cluster=False,
            row_cluster=True,
            fmt=".3f",
            center=1.0,
            cmap='PiYG_r',
            linewidth=0.1,
            linecolor='black',
            tree_kws=dict(linewidths=1.5),
            dendrogram_ratio=(0.6, 0.0),
            cbar_pos=(0.15, 1.06, 0.9, 0.04),
            cbar_kws={"orientation": "horizontal"},
            annot_kws={"size": 55 / np.sqrt(max(df_states_aerr_mean.shape))},
            figsize=(4, (0.25 + 0.03 * len(states)) * 10)
        )
        clustermap.ax_heatmap.set_xlabel('DunedinPACE', fontsize=20)
        clustermap.ax_heatmap.set_xticklabels("", fontsize = 18)
        clustermap.ax_heatmap.set_ylabel('Countries', fontsize=20)
        clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize = 18)
        clustermap.ax_cbar.set_title(r"$\langle$ Pace of Aging $\rangle$", size=20)
        clustermap.ax_cbar.tick_params(labelsize=18)
        for spine in clustermap.ax_cbar.spines.values():
            spine.set(visible=True, lw=0.25, edgecolor="black")
        clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        clustermap_order_pace = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
        for tick_label in clustermap.ax_heatmap.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/clustermap_pace_mean.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/clustermap_pace_mean.pdf", bbox_inches='tight')
        plt.close(clustermap.fig)
        
        fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(states)))
        sns.set_theme(style='whitegrid')
        violin = sns.violinplot(
            data=df_tissue,
            x='DunedinPACE',
            y='State',
            orient='h',
            inner='box',
            cut=0,
            scale='width',
            order=clustermap_order_pace,
            palette=colors_states,
            ax=ax
        )
        ax.set_xlabel('DunedinPace', fontsize=20)
        ax.set_ylabel('Countries', fontsize=20)
        ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize = 23)
        ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
        plt.savefig(f"{path_save}/violin_pace_for_clustermap.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/violin_pace_for_clustermap.pdf", bbox_inches='tight')
        plt.close(fig)
        
        path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/states/epi_ests"
        pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
        dfs_states_states_stat = {}
        for epi_est in ages + [pace]:
            dfs_states_states_stat[epi_est] = pd.DataFrame(index=states, columns=states, data=np.zeros(shape=(len(states), len(states))))
            if epi_est == pace:
                col = epi_est
            else:
                col = f"{epi_est}Acc"
            for state_1_id, state_1 in enumerate(states):
                vals_1 = df_tissue.loc[df_tissue['State'] == state_1,  col].values
                for state_2_id in range(state_1_id, len(states)):
                    state_2 = states[state_2_id]
                    vals_2 = df_tissue.loc[df_tissue['State'] == state_2, col].values
                    if state_1 != state_2:
                        _, pval = mannwhitneyu(vals_1, vals_2, alternative='two-sided')
                        diff = np.mean(vals_2) - np.mean(vals_1)
                        dfs_states_states_stat[epi_est].at[state_1, state_2] = diff
                        dfs_states_states_stat[epi_est].at[state_2, state_1] = pval
                    else:
                        dfs_states_states_stat[epi_est].at[state_1, state_2]  = np.nan
            selection = np.tri(len(states), len(states), -1, dtype=np.bool)
            df_fdr = dfs_states_states_stat[epi_est].where(selection).stack().reset_index()
            df_fdr.columns = ['row', 'col', 'pval']
            _, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
            nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
            df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
            df_fdr['pval_fdr_bh_log'] = -np.log10(df_fdr.loc[:, 'pval_fdr_bh'].values)
            for line_id in range(df_fdr.shape[0]):
                dfs_states_states_stat[epi_est].loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = df_fdr.at[line_id, 'pval_fdr_bh_log']
            dfs_states_states_stat[epi_est].to_excel(f"{path_save}/{epi_est}_stat.xlsx", index_label="State")
            
            fig, ax = plt.subplots(figsize=(4.2 + 0.23 * len(states), 0.8 + 0.2 * len(states)))
            sns.set_theme(style='whitegrid')
            cmap_triu = plt.get_cmap("seismic").copy()
            heatmap_diff = sns.heatmap(
                dfs_states_states_stat[epi_est],
                mask=np.tri(len(states), len(states), -1, dtype=np.bool),
                annot=True,
                fmt=".2f",
                center=0.0,
                cmap=cmap_triu,
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 25 / np.sqrt(max(df_states_aerr_mean.shape))},
                ax=ax
            )
            if epi_est == pace:
                ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$ Difference", size=13)
            else:
                ax.figure.axes[-1].set_ylabel(r"$\langle$ Age Acceleration $\rangle$ Difference", size=13)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            cmap_tril = plt.get_cmap("cool").copy()
            cmap_tril.set_under('black')
            heatmap_pval = sns.heatmap(
                dfs_states_states_stat[epi_est],
                mask=np.tri(len(states), len(states), -1, dtype=np.bool).T,
                annot=True,
                fmt=".1f",
                vmin=-np.log10(0.05),
                cmap=cmap_tril,
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 25 / np.sqrt(max(df_states_aerr_mean.shape))},
                ax=ax
            )
            ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$", size=13)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            ax.set_xlabel('', fontsize=16)
            ax.set_ylabel('', fontsize=16)
            ax.set_title(epi_est, fontsize=16)
            ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_xticklabels():
                tick_label.set_color(colors_states[tick_label.get_text()])
                ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_states[tick_label.get_text()])
                ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            plt.savefig(f"{path_save}/{epi_est}_stat.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/{epi_est}_stat.pdf", bbox_inches='tight')
            plt.close(fig)
        
        df_states_states_stat = pd.DataFrame(index=states, columns=states, data=np.zeros(shape=(len(states), len(states))))
        for state_1_id, state_1 in enumerate(states):
            for state_2_id in range(state_1_id, len(states)):
                state_2 = states[state_2_id]
                for epi_est in ages + [pace]:
                    pval = dfs_states_states_stat[epi_est].at[state_2, state_1]
                    diff = dfs_states_states_stat[epi_est].at[state_1, state_2]
                    if diff > 0:
                        df_states_states_stat.at[state_1, state_2] += 1
                    else:
                        df_states_states_stat.at[state_1, state_2] -= 1
                    df_states_states_stat.at[state_2, state_1] += pval / (len(ages + [pace]))
        df_states_states_stat.to_excel(f"{path_save}/global_stat.xlsx", index_label="State")
        fig, ax = plt.subplots(figsize=(4.2 + 0.23 * len(states), 0.8 + 0.2 * len(states)))
        sns.set_theme(style='whitegrid')
        cmap_triu = plt.get_cmap("seismic").copy()
        sns.heatmap(
            df_states_states_stat,
            mask=np.tri(len(states), len(states), -1, dtype=np.bool),
            annot=True,
            cmap=cmap_triu,
            center=0.0,
            linewidth=0.1,
            linecolor='black',
            annot_kws={"size": 25 / np.sqrt(max(df_states_aerr_mean.shape))},
            ax=ax
        )
        ax.figure.axes[-1].set_ylabel(r"Aggregated Sign of $\langle$ EAA $\rangle$ Difference", size=10)
        for spine in ax.figure.axes[-1].spines.values():
            spine.set(visible=True, lw=0.25, edgecolor="black")
        cmap_tril = plt.get_cmap("cool").copy()
        cmap_tril.set_under('black')
        sns.heatmap(
            df_states_states_stat,
            mask=np.tri(len(states), len(states), -1, dtype=np.bool).T,
            annot=True,
            fmt=".1f",
            vmin=-np.log10(0.05),
            cmap=cmap_tril,
            linewidth=0.1,
            linecolor='black',
            annot_kws={"size": 25 / np.sqrt(max(df_states_aerr_mean.shape))},
            ax=ax
        )
        ax.figure.axes[-1].set_ylabel(r"$\langle -\log_{10}(\mathrm{p-value}) \rangle$", size=13)
        for spine in ax.figure.axes[-1].spines.values():
            spine.set(visible=True, lw=0.25, edgecolor="black")
        ax.set_xlabel('', fontsize=16)
        ax.set_ylabel('', fontsize=16)
        ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in ax.get_xticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
            ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        for tick_label in ax.get_yticklabels():
            tick_label.set_color(colors_states[tick_label.get_text()])
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
        plt.savefig(f"{path_save}/global_stat.png", bbox_inches='tight', dpi=200)
        plt.savefig(f"{path_save}/global_stat.pdf", bbox_inches='tight')
        plt.close(fig)
        

## Plot by states

### Plot groups

In [ ]:
for tissue in tissues:
    print(tissue)
    df_tissue = df.loc[df['Tissue'] == tissue, :]

    for state in df_states_by_tissue[tissue].index.values:
        
        df_state = df_tissue.loc[df_tissue['State'] == state, :]
        df_states_group = dfs_states_group_by_tissue[tissue][state]
        groups = df_states_group.index.values
        
        if len(groups) > 1:
            
            colors_groups = {group: colors_global[group] for group in groups}
            
            # Plot sex-specificity
            df_ss = pd.DataFrame(index=groups, columns=['F older M', 'M older F', 'No significance'])
            for group in groups:
                df_group = df_tissue.loc[df_tissue['Group'] == group,  :]
                path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_group/{group}/sex_specific"
                pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
                
                df_stat_ss = pd.DataFrame(index=ages + [pace])
                for epi_est in ages + [pace]:
                    if epi_est == pace:
                        col = epi_est
                    else:
                        col = f"{epi_est}Acc"
                    vals_f = df_group.loc[df_group['Sex'] == 'F',  col].values
                    vals_m = df_group.loc[df_group['Sex'] == 'M',  col].values
                    if len(vals_f) > 0 and len(vals_m) > 0:
                        _, df_stat_ss.at[epi_est, 'pval'] = mannwhitneyu(vals_f, vals_m, alternative='two-sided')
                        if np.mean(vals_f) > np.mean(vals_m):
                            df_stat_ss.at[epi_est, 'older'] = 'F'
                        else:
                            df_stat_ss.at[epi_est, 'older'] = 'M'
                    else:
                        df_stat_ss.at[epi_est, 'pval'] = 1.0
                _, df_stat_ss['pval_fdr_bh'], _, _ = multipletests(df_stat_ss.loc[:, 'pval'].values, 0.05, method='fdr_bh')
                nzmin = df_stat_ss['pval_fdr_bh'][df_stat_ss['pval_fdr_bh'].gt(0)].min(0) * 0.5
                df_stat_ss['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
                df_stat_ss['pval_fdr_bh_log'] = -np.log10(df_stat_ss.loc[:, 'pval_fdr_bh'].values)
                df_stat_ss.to_excel(f"{path_save}/stat.xlsx")
                
                if df_group[df_group['Sex'] == 'F'].shape[0] > 0 and df_group[df_group['Sex'] == 'M'].shape[0] > 0:
                    df_ss.at[group, 'Insignificant / One sex'] = df_stat_ss[df_stat_ss['pval_fdr_bh'] >= 0.05].shape[0]
                    df_ss.at[group, 'F older M'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'F')].shape[0]
                    df_ss.at[group, 'M older F'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'M')].shape[0]
                else:
                    df_ss.at[group, 'Insignificant / One sex'] = len(ages) + 1
                    df_ss.at[group, 'F older M'] = 0
                    df_ss.at[group, 'M older F'] = 0
                    
                fig = make_subplots(rows=2, cols=1, row_heights=[11.0, 1.0], vertical_spacing=0.1)
                age_vio_labels = {}
                ages_ordered = [
                    'Hannum',
                    'PC-Hannum',
                    'Horvath',
                    'PC-Horvath',
                    'SkinBloodAge',
                    'PC-SkinBloodAge',
                    'PhenoAge',
                    'PC-PhenoAge',
                    'GrimAge',
                    'PC-GrimAge',
                    'GrimAge2'
                ][::-1]
                for epi_est_id, epi_est in enumerate(ages_ordered):
                    vals_f = df_group.loc[df_group['Sex'] == 'F',  f"{epi_est}Acc"].values
                    vals_m = df_group.loc[df_group['Sex'] == 'M',  f"{epi_est}Acc"].values
                    pval = df_stat_ss.at[epi_est, 'pval_fdr_bh']
                    age_vio_labels[epi_est] = f"{epi_est}<br>p-value: {pval:0.2e}"
                    if len(vals_f) > 0 and len(vals_m) > 0:
                        fig.add_trace(
                            go.Violin(
                                y=[epi_est_id] * len(vals_m),
                                x=vals_m,
                                name=age_vio_labels[epi_est],
                                box_visible=True,
                                box_width=0.8,
                                meanline_visible=True,
                                showlegend=False,
                                line_color='black',
                                fillcolor=colors_sex['M'],
                                marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                                points=False,
                                bandwidth=np.ptp(vals_m) / 25,
                                opacity=0.8,
                                legendgroup=age_vio_labels[epi_est],
                                scalegroup=age_vio_labels[epi_est],
                                side='negative',
                                orientation='h',
                                scalemode="width",
                                pointpos=-1.5,
                            ),
                            row=1,
                            col=1
                        )
                        fig.add_trace(
                            go.Violin(
                                y=[epi_est_id] * len(vals_f),
                                x=vals_f,
                                name=age_vio_labels[epi_est],
                                box_visible=True,
                                box_width=0.8,
                                meanline_visible=True,
                                showlegend=False,
                                line_color='black',
                                fillcolor=colors_sex['F'],
                                marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                                points=False,
                                bandwidth=np.ptp(vals_f) / 25,
                                opacity=0.8,
                                legendgroup=age_vio_labels[epi_est],
                                scalegroup=age_vio_labels[epi_est],
                                scalemode="width",
                                side='positive',
                                orientation='h',
                                pointpos=1.5,
                            ),
                            row=1,
                            col=1
                        )
                fig.update_layout(template="none")
                fig.update_xaxes(
                    title_text="Epigenetic Age Acceleration",
                    tickfont=dict(size=26),
                    titlefont=dict(size=26),
                    showgrid=True,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=1,
                    col=1
                )
                fig.update_yaxes(
                    tickmode='array',
                    tickvals=list(range(len(ages_ordered))),
                    ticktext=[color_tick('black', age_vio_labels[x]) for x in ages_ordered],
                    tickfont=dict(size=25),
                    showgrid=False,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=1,
                    col=1
                )
                fig.update_yaxes(autorange=False, range=[-0.5, len(ages) - 0.5], row=1, col=1)
                fig.update_xaxes(autorange=True, row=1, col=1)
            
                vals_f = df_group.loc[df_group['Sex'] == 'F', pace].values
                vals_m = df_group.loc[df_group['Sex'] == 'M', pace].values
                pval = df_stat_ss.at[pace, 'pval_fdr_bh']
                age_vio_labels[pace] = f"{pace}<br>p-value: {pval:0.2e}"
                if len(vals_f) > 0 and len(vals_m) > 0:
                    fig.add_trace(
                        go.Violin(
                            y=[0] * len(vals_m),
                            x=vals_m,
                            name=age_vio_labels[pace],
                            box_visible=True,
                            box_width=0.8,
                            meanline_visible=True,
                            showlegend=False,
                            line_color='black',
                            fillcolor=colors_sex['M'],
                            marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                            points=False,
                            bandwidth=np.ptp(vals_m) / 25,
                            opacity=0.8,
                            legendgroup=age_vio_labels[pace],
                            scalegroup=age_vio_labels[pace],
                            side='negative',
                            orientation='h',
                            scalemode="width",
                            pointpos=-1.5,
                        ),
                        row=2,
                        col=1
                    )
                    fig.add_trace(
                        go.Violin(
                            y=[0] * len(vals_f),
                            x=vals_f,
                            name=age_vio_labels[pace],
                            box_visible=True,
                            box_width=0.8,
                            meanline_visible=True,
                            showlegend=False,
                            line_color='black',
                            fillcolor=colors_sex['F'],
                            marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                            points=False,
                            bandwidth=np.ptp(vals_f) / 25,
                            opacity=0.8,
                            legendgroup=age_vio_labels[pace],
                            scalegroup=age_vio_labels[pace],
                            scalemode="width",
                            side='positive',
                            orientation='h',
                            pointpos=1.5,
                        ),
                        row=2,
                        col=1
                    )
                fig.update_xaxes(
                    title_text="Pace of Aging",
                    tickfont=dict(size=26),
                    titlefont=dict(size=26),
                    showgrid=True,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=2,
                    col=1
                )
                fig.update_yaxes(
                    tickmode='array',
                    tickvals=[0],
                    ticktext=[color_tick('black', age_vio_labels[pace])],
                    tickfont=dict(size=25),
                    showgrid=False,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=2,
                    col=1
                )
                fig.update_yaxes(autorange=False, range=[-0.5, 0.5], row=2, col=1)
                fig.update_xaxes(autorange=True, row=2, col=1)
                
                fig.update_layout(
                    title=dict(xref='paper', x=1.0),
                    legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.01,
                        xanchor="left",
                        x=0.0001,
                        itemsizing='constant',
                        font_size=22
                    ),
                )
                fig.update_layout(
                    violingap=0.05,
                    violingroupgap=0.05,
                    height=140 * (len(ages) + 1),
                    width=1050,
                    margin=go.layout.Margin(
                        l=300,
                        r=30,
                        b=110,
                        t=30,
                        pad=0,
                    )
                )
                fig.write_image(f"{path_save}/violins.png", scale=2)
                fig.write_image(f"{path_save}/violins.pdf", format="pdf")
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_group"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)    
            df_ss.to_excel(f"{path_save}/ss.xlsx", index_label='Group')
            df_ss['Group'] = df_ss.index.values
            df_ss_melt = pd.melt(df_ss, id_vars=['Group'], value_vars=['F older M', 'M older F', 'Insignificant / One sex'], var_name='EAA type', value_name='Number of EAA estimators')
            fig, ax = plt.subplots(figsize=(3, 1.5 + 0.8 * len(groups)))
            sns.set_theme(style='whitegrid')
            barplot = sns.barplot(
                data=df_ss_melt,
                y='Group',
                hue='EAA type',
                x='Number of EAA estimators',
                edgecolor='black',
                palette={'F older M': 'red', 'M older F': 'blue', 'Insignificant / One sex': 'gray'},
                ax=ax
            )
            ax.set_xlabel('Number of EAA estimators', fontsize=20)
            ax.set_ylabel('', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            for container in ax.containers:
                ax.bar_label(container, fmt='%d', fontsize=12)
            sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1))
            plt.savefig(f"{path_save}/ss_barplot.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/ss_barplot.pdf", bbox_inches='tight')
            plt.close(fig)
            
            n_rows = 2
            n_cols = 6
            fig_width = 15
            fig_height = 8
            low_percent = 0.005
            hgh_percent = 0.995
            ptp_shift = 0.1
    
            for group in groups:
                df_group = df_state.loc[df_state['Group'] == group,  :]
    
                x_min = df_group['Age'].min()
                x_max = df_group['Age'].max()
                x_ptp = x_max - x_min
    
                y_min = df_group[list(ages)].min().min()
                y_max = df_group[list(ages)].max().max()
                y_ptp = y_max - y_min
    
                y_percentiles = df_group[ages].quantile([low_percent, hgh_percent])
                y_low = np.min(y_percentiles.loc[low_percent, :].values)
                y_hgh = np.max(y_percentiles.loc[hgh_percent, :].values)
                
                path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_group/{group}/sex_specific"
                pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
                
                fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
                sns.set_theme(style='whitegrid')
                
                for epi_est, params in ages_kde.items():
                    if epi_est == 'PC-GrimAge2':
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        axs[params['row_id'], params['col_id']].axis('off')
                    else:
                        for sex in ['M', 'F']:
                            try:
                                kdeplot = sns.kdeplot(
                                    data=df_group.loc[df_group['Sex'] == sex, :],
                                    x='Age',
                                    y=epi_est,
                                    fill=True,
                                    cbar=False,
                                    alpha=0.6,
                                    cut=0,
                                    color=colors_sex[sex],
                                    # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                                    legend=False,
                                    ax=axs[params['row_id'], params['col_id']]
                                )
                            except linalg.LinAlgError:
                                print(f"LinAlgError: {tissue} {state} {group}")

                        points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
                        axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)
                        
                        axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
                        axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
                        axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        if params['x_label'] == '':
                            axs[params['row_id'], params['col_id']].set_xticklabels([])
                
                fig.tight_layout()    
                plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
                plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
                plt.close(fig)
                
                path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_group/{group}"
                pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
                
                fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
                sns.set_theme(style='whitegrid')
    
                for epi_est, params in ages_kde.items():
                    if epi_est == 'PC-GrimAge2':
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        axs[params['row_id'], params['col_id']].axis('off')
                    else:
                        kdeplot = sns.kdeplot(
                            data=df_group,
                            x='Age',
                            y=epi_est,
                            fill=True,
                            cbar=False,
                            color=colors_groups[group],
                            cut=0,
                            # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                            legend=False,
                            ax=axs[params['row_id'], params['col_id']]
                        )
                        regplot = sns.regplot(
                            data=df_group,
                            x='Age',
                            y=epi_est,
                            scatter=False,
                            color='black',
                            truncate=True,
                            ax=axs[params['row_id'], params['col_id']]
                        )
                        points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
                        axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)
                        
                        if df_group.shape[0] > 1:
                            corr, _ = stats.pearsonr(df_group['Age'].values, df_group[epi_est].values)
                            mae = mean_absolute_error(df_group['Age'].values, df_group[epi_est].values)
                            label = r'$\rho$ = ' + f"{corr:0.2f}"
                            axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.10), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)
                            label = f"MAE = {mae:0.2f}"
                            axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.02), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)
    
                        axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
                        axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
                        axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        if params['x_label'] == '':
                            axs[params['row_id'], params['col_id']].set_xticklabels([])
    
                fig.tight_layout()    
                plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
                plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
                plt.close(fig)
            
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/groups"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
            
            hist_bins = np.linspace(0, 110, 12)
            fig, ax = plt.subplots()
            sns.set_theme(style='whitegrid')
            histplot = sns.histplot(
                data=df_state,
                x=f"Age",
                bins=hist_bins,
                edgecolor='k',
                linewidth=1,
                hue_order=list(colors_groups.keys())[::-1],
                hue="Group",
                palette=colors_groups,
                multiple="stack",
                ax=ax
            )
            plt.savefig(f"{path_save}/histplot.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/histplot.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(3, 0.5 + 0.8 * len(groups)))
            sns.set_theme(style='whitegrid')
            barplot = sns.barplot(
                data=dfs_states_group_by_tissue[tissue][state],
                y=dfs_states_group_by_tissue[tissue][state].index.values,
                hue=dfs_states_group_by_tissue[tissue][state].index.values,
                x=dfs_states_group_by_tissue[tissue][state]['Group'].values,
                edgecolor='black',
                palette=colors_groups,
                dodge=False,
                ax=ax
            )
            ax.set_xlabel('Number of samples', fontsize=20)
            ax.set_ylabel('', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.get_legend().remove()
            for container in ax.containers:
                ax.bar_label(container, fmt='%d', fontsize=18)
            plt.savefig(f"{path_save}/barplot.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/barplot.pdf", bbox_inches='tight')
            plt.close(fig)
            
            df_groups_aerr_mean = pd.DataFrame(index=groups, columns=ages, data=np.zeros(shape=(len(groups), len(ages))))
            df_groups_pace_mean = pd.DataFrame(index=groups, columns=[pace], data=np.zeros(shape=(len(groups), 1)))
            for group in groups:
                vals = df_state.loc[df_state['Group'] == group, pace].values
                df_groups_pace_mean.at[group, pace] = np.mean(vals)
                for age_type in ages:
                    vals = df_state.loc[df_state['Group'] == group, f"{age_type}Acc"].values
                    df_groups_aerr_mean.at[group, age_type] = np.mean(vals)
            df_groups_aerr_mean.to_excel(f"{path_save}/aerr_mean.xlsx", index_label="Group")
            df_groups_pace_mean.to_excel(f"{path_save}/pace_mean.xlsx", index_label="Group")
            
            fig, ax = plt.subplots(figsize=(2.7 + 0.375 * len(ages), 1.3 + 0.15 * len(groups)))
            sns.set_theme(style='whitegrid')
            heatmap = sns.heatmap(
                df_groups_aerr_mean,
                annot=True,
                fmt=".1f",
                center=0.0,
                cmap='seismic',
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 35 / np.sqrt(max(df_groups_aerr_mean.shape))},
                ax=ax,
            )
            ax.set_xlabel('Epigenetic Age', fontsize=16)
            ax.set_ylabel('Groups', fontsize=16)
            ax.figure.axes[-1].set_ylabel(r"$\langle$ Acceleration $\rangle$", size=16)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            heatmap_order_ages = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/heatmap_aerr_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/heatmap_aerr_mean.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(6, 0.5 + 0.8 * len(groups)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='Age',
                y='Group',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=heatmap_order_ages,
                palette=colors_groups,
                ax=ax
            )
            ax.set_xlabel('Chronological Age', fontsize=20)
            ax.set_ylabel('Groups', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 21)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_age_for_heatmap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_age_for_heatmap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            if len(groups) > 2:
                figsize_shift_x = 0.35
                row_cluster = True
            else:
                figsize_shift_x = 0.125
                row_cluster = False
            sns.set_theme(style='whitegrid')
            clustermap = sns.clustermap(
                df_groups_aerr_mean,
                annot=True,
                col_cluster=True,
                row_cluster=row_cluster,
                fmt=".1f",
                center=0.0,
                cmap='seismic',
                linewidth=0.1,
                linecolor='black',
                tree_kws=dict(linewidths=1.5),
                annot_kws={"size": 55 / np.sqrt(max(df_groups_aerr_mean.shape))},
                figsize=((figsize_shift_x + 0.065 * len(ages)) * 10, (0.25 + 0.030 * len(groups)) * 10)
            )
            clustermap.ax_heatmap.set_xlabel('Epigenetic Age', fontsize=20)
            clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xmajorticklabels(), fontsize=18)
            clustermap.ax_heatmap.set_ylabel('Groups', fontsize=20)
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize=20, rotation=0)
            clustermap.ax_cbar.set_ylabel(r"$\langle$ Acceleration $\rangle$", size=20)
            clustermap.ax_cbar.tick_params(labelsize=18)
            clustermap_order_ages = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
            for spine in clustermap.ax_cbar.spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in clustermap.ax_heatmap.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/clustermap_aerr_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/clustermap_aerr_mean.pdf", bbox_inches='tight')
            plt.close(clustermap.fig)
            
            fig, ax = plt.subplots(figsize=(6, 0.5 + 0.8 * len(groups)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='Age',
                y='Group',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=clustermap_order_ages,
                palette=colors_groups,
                ax=ax
            )
            ax.set_xlabel('Chronological Age', fontsize=20)
            ax.set_ylabel('Groups', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 21)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_age_for_clustermap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_age_for_clustermap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(1.5, 1.8 + 0.15 * len(groups)))
            sns.set_theme(style='whitegrid')
            heatmap = sns.heatmap(
                df_groups_pace_mean,
                annot=True,
                fmt=".3f",
                center=1.0,
                cmap='PiYG_r',
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 35 / np.sqrt(max(df_groups_aerr_mean.shape))},
                ax=ax
            )
            ax.set_xticklabels([''])
            ax.set_xlabel('DunedinPACE', fontsize=16)
            ax.set_ylabel('Groups', fontsize=16)
            ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$", size=16)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            heatmap_order_pace = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/heatmap_pace_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/heatmap_pace_mean.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(6, 0.5 + 0.8 * len(groups)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='DunedinPACE',
                y='Group',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=heatmap_order_pace,
                palette=colors_groups,
                ax=ax
            )
            ax.set_xlabel('DunedinPace', fontsize=20)
            ax.set_ylabel('Groups', fontsize=20)
            ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 21)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_pace_for_heatmap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_pace_for_heatmap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            if len(groups) > 2:
                figsize_shift_y = 0.25
                row_cluster = True
            else:
                figsize_shift_y = 0.25
                row_cluster = False
            sns.set_theme(style='whitegrid')
            clustermap = sns.clustermap(
                df_groups_pace_mean,
                annot=True,
                col_cluster=False,
                row_cluster=row_cluster,
                fmt=".3f",
                center=1.0,
                cmap='PiYG_r',
                linewidth=0.1,
                linecolor='black',
                tree_kws=dict(linewidths=1.5),
                dendrogram_ratio=(0.6, 0.0),
                cbar_pos=(0.15, 1.06, 0.9, 0.04),
                cbar_kws={"orientation": "horizontal"},
                annot_kws={"size": 55 / np.sqrt(max(df_groups_aerr_mean.shape))},
                figsize=(3.5, (figsize_shift_y + 0.0175 * len(groups)) * 10)
            )
            clustermap.ax_heatmap.set_xlabel('DunedinPACE', fontsize=20)
            clustermap.ax_heatmap.set_xticklabels("", fontsize = 18)
            clustermap.ax_heatmap.set_ylabel('Groups', fontsize=20)
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize=21, rotation=0)
            clustermap.ax_cbar.set_title(r"$\langle$ Pace of Aging $\rangle$", size=20)
            clustermap.ax_cbar.tick_params(labelsize=18)
            for spine in clustermap.ax_cbar.spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            clustermap_order_pace = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
            for tick_label in clustermap.ax_heatmap.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/clustermap_pace_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/clustermap_pace_mean.pdf", bbox_inches='tight')
            plt.close(clustermap.fig)
            
            fig, ax = plt.subplots(figsize=(6, 0.5 + 0.8 * len(groups)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='DunedinPACE',
                y='Group',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=clustermap_order_pace,
                palette=colors_groups,
                ax=ax
            )
            ax.set_xlabel('DunedinPace', fontsize=20)
            ax.set_ylabel('Groups', fontsize=20)
            ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 21)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_pace_for_clustermap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_pace_for_clustermap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/groups/epi_ests"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
            dfs_groups_groups_stat = {}
            for epi_est in ages + [pace]:
                dfs_groups_groups_stat[epi_est] = pd.DataFrame(index=groups, columns=groups, data=np.zeros(shape=(len(groups), len(groups))))
                if epi_est == pace:
                    col = epi_est
                else:
                    col = f"{epi_est}Acc"
                for group_1_id, group_1 in enumerate(groups):
                    vals_1 = df_state.loc[df_state['Group'] == group_1,  col].values
                    for group_2_id in range(group_1_id, len(groups)):
                        group_2 = groups[group_2_id]
                        vals_2 = df_state.loc[df_state['Group'] == group_2, col].values
                        if group_1 != group_2:
                            _, pval = mannwhitneyu(vals_1, vals_2, alternative='two-sided')
                            diff = np.mean(vals_2) - np.mean(vals_1)
                            dfs_groups_groups_stat[epi_est].at[group_1, group_2] = diff
                            dfs_groups_groups_stat[epi_est].at[group_2, group_1] = pval
                        else:
                            dfs_groups_groups_stat[epi_est].at[group_1, group_2]  = np.nan
                selection = np.tri(len(groups), len(groups), -1, dtype=np.bool)
                df_fdr = dfs_groups_groups_stat[epi_est].where(selection).stack().reset_index()
                df_fdr.columns = ['row', 'col', 'pval']
                _, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
                nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
                df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
                df_fdr['pval_fdr_bh_log'] = -np.log10(df_fdr.loc[:, 'pval_fdr_bh'].values)
                for line_id in range(df_fdr.shape[0]):
                    dfs_groups_groups_stat[epi_est].loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = df_fdr.at[line_id, 'pval_fdr_bh_log']
                dfs_groups_groups_stat[epi_est].to_excel(f"{path_save}/{epi_est}_stat.xlsx", index_label="groupName")
                
                fig, ax = plt.subplots(figsize=(4.8 + 0.2 * len(groups), 0.8 + 0.2 * len(groups)))
                sns.set_theme(style='whitegrid')
                cmap_triu = plt.get_cmap("seismic").copy()
                heatmap_diff = sns.heatmap(
                    dfs_groups_groups_stat[epi_est],
                    mask=np.tri(len(groups), len(groups), -1, dtype=np.bool),
                    annot=True,
                    fmt=".2f",
                    center=0.0,
                    cmap=cmap_triu,
                    linewidth=0.1,
                    linecolor='black',
                    annot_kws={"size": 25 / np.sqrt(len(groups))},
                    ax=ax
                )
                if epi_est == pace:
                    ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$ Difference", size=13)
                else:
                    ax.figure.axes[-1].set_ylabel(r"$\langle$ Age Acceleration $\rangle$ Difference", size=13)
                for spine in ax.figure.axes[-1].spines.values():
                    spine.set(visible=True, lw=0.25, edgecolor="black")
                cmap_tril = plt.get_cmap("cool").copy()
                cmap_tril.set_under('black')
                heatmap_pval = sns.heatmap(
                    dfs_groups_groups_stat[epi_est],
                    mask=np.tri(len(groups), len(groups), -1, dtype=np.bool).T,
                    annot=True,
                    fmt=".1f",
                    vmin=-np.log10(0.05),
                    cmap=cmap_tril,
                    linewidth=0.1,
                    linecolor='black',
                    annot_kws={"size": 25 / np.sqrt(len(groups))},
                    ax=ax
                )
                ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$", size=13)
                for spine in ax.figure.axes[-1].spines.values():
                    spine.set(visible=True, lw=0.25, edgecolor="black")
                ax.set_xlabel('', fontsize=16)
                ax.set_ylabel('', fontsize=16)
                ax.set_title(epi_est, fontsize=16)
                ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
                for tick_label in ax.get_xticklabels():
                    tick_label.set_color(colors_groups[tick_label.get_text()])
                    ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
                for tick_label in ax.get_yticklabels():
                    tick_label.set_color(colors_groups[tick_label.get_text()])
                    ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
                plt.savefig(f"{path_save}/{epi_est}_stat.png", bbox_inches='tight', dpi=200)
                plt.savefig(f"{path_save}/{epi_est}_stat.pdf", bbox_inches='tight')
                plt.close(fig)
                
            df_groups_groups_stat = pd.DataFrame(index=groups, columns=groups, data=np.zeros(shape=(len(groups), len(groups))))
            for group_1_id, group_1 in enumerate(groups):
                for group_2_id in range(group_1_id, len(groups)):
                    group_2 = groups[group_2_id]
                    for epi_est in ages + [pace]:
                        pval = dfs_groups_groups_stat[epi_est].at[group_2, group_1]
                        diff = dfs_groups_groups_stat[epi_est].at[group_1, group_2]
                        if diff > 0:
                            df_groups_groups_stat.at[group_1, group_2] += 1
                        else:
                            df_groups_groups_stat.at[group_1, group_2] -= 1
                        df_groups_groups_stat.at[group_2, group_1] += pval / (len(ages + [pace]))
            df_groups_groups_stat.to_excel(f"{path_save}/global_stat.xlsx", index_label="Group")
            fig, ax = plt.subplots(figsize=(4.8 + 0.2 * len(groups), 0.8 + 0.2 * len(groups)))
            sns.set_theme(style='whitegrid')
            cmap_triu = plt.get_cmap("seismic").copy()
            sns.heatmap(
                df_groups_groups_stat,
                mask=np.tri(len(groups), len(groups), -1, dtype=np.bool),
                annot=True,
                cmap=cmap_triu,
                center=0.0,
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 35 / np.sqrt(max(df_groups_aerr_mean.shape))},
                ax=ax
            )
            ax.figure.axes[-1].set_ylabel(r"Aggregated Sign of $\langle$ EAA $\rangle$ Difference", size=13)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            cmap_tril = plt.get_cmap("cool").copy()
            cmap_tril.set_under('black')
            sns.heatmap(
                df_groups_groups_stat,
                mask=np.tri(len(groups), len(groups), -1, dtype=np.bool).T,
                annot=True,
                fmt=".1f",
                vmin=-np.log10(0.05),
                cmap=cmap_tril,
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 35 / np.sqrt(max(df_groups_aerr_mean.shape))},
                ax=ax
            )
            ax.figure.axes[-1].set_ylabel(r"$\langle -\log_{10}(\mathrm{p-value}) \rangle$", size=13)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            ax.set_xlabel('', fontsize=16)
            ax.set_ylabel('', fontsize=16)
            ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_xticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
                ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_groups[tick_label.get_text()])
                ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            plt.savefig(f"{path_save}/global_stat.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/global_stat.pdf", bbox_inches='tight')
            plt.close(fig)
            

### Plot gses

In [ ]:
for tissue in tissues:
    print(tissue)
    df_tissue = df.loc[df['Tissue'] == tissue, :]

    for state in df_states_by_tissue[tissue].index.values:
        
        df_state = df_tissue.loc[df_tissue['State'] == state, :]
        df_states_gse = dfs_states_gse_by_tissue[tissue][state]
        gses = df_states_gse.index.values
        
        if len(gses) > 1:
            
            colors_gses = {gse: colors_global[gse] for gse in gses}
            
            # Plot sex-specificity
            df_ss = pd.DataFrame(index=gses, columns=['F older M', 'M older F', 'No significance'])
            for gse in gses:
                df_gse = df_tissue.loc[df_tissue['GSE'] == gse,  :]
                path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_gse/{gse}/sex_specific"
                pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
                
                df_stat_ss = pd.DataFrame(index=ages + [pace])
                for epi_est in ages + [pace]:
                    if epi_est == pace:
                        col = epi_est
                    else:
                        col = f"{epi_est}Acc"
                    vals_f = df_gse.loc[df_gse['Sex'] == 'F',  col].values
                    vals_m = df_gse.loc[df_gse['Sex'] == 'M',  col].values
                    if len(vals_f) > 0 and len(vals_m) > 0:
                        _, df_stat_ss.at[epi_est, 'pval'] = mannwhitneyu(vals_f, vals_m, alternative='two-sided')
                        if np.mean(vals_f) > np.mean(vals_m):
                            df_stat_ss.at[epi_est, 'older'] = 'F'
                        else:
                            df_stat_ss.at[epi_est, 'older'] = 'M'
                    else:
                        df_stat_ss.at[epi_est, 'pval'] = 1.0
                _, df_stat_ss['pval_fdr_bh'], _, _ = multipletests(df_stat_ss.loc[:, 'pval'].values, 0.05, method='fdr_bh')
                nzmin = df_stat_ss['pval_fdr_bh'][df_stat_ss['pval_fdr_bh'].gt(0)].min(0) * 0.5
                df_stat_ss['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
                df_stat_ss['pval_fdr_bh_log'] = -np.log10(df_stat_ss.loc[:, 'pval_fdr_bh'].values)
                df_stat_ss.to_excel(f"{path_save}/stat.xlsx")
                
                if df_gse[df_gse['Sex'] == 'F'].shape[0] > 0 and df_gse[df_gse['Sex'] == 'M'].shape[0] > 0:
                    df_ss.at[gse, 'Insignificant / One sex'] = df_stat_ss[df_stat_ss['pval_fdr_bh'] >= 0.05].shape[0]
                    df_ss.at[gse, 'F older M'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'F')].shape[0]
                    df_ss.at[gse, 'M older F'] = df_stat_ss[(df_stat_ss['pval_fdr_bh'] < 0.05) & (df_stat_ss['older'] == 'M')].shape[0]
                else:
                    df_ss.at[gse, 'Insignificant / One sex'] = len(ages) + 1
                    df_ss.at[gse, 'F older M'] = 0
                    df_ss.at[gse, 'M older F'] = 0
                    
                fig = make_subplots(rows=2, cols=1, row_heights=[11.0, 1.0], vertical_spacing=0.1)
                age_vio_labels = {}
                ages_ordered = [
                    'Hannum',
                    'PC-Hannum',
                    'Horvath',
                    'PC-Horvath',
                    'SkinBloodAge',
                    'PC-SkinBloodAge',
                    'PhenoAge',
                    'PC-PhenoAge',
                    'GrimAge',
                    'PC-GrimAge',
                    'GrimAge2'
                ][::-1]
                for epi_est_id, epi_est in enumerate(ages_ordered):
                    vals_f = df_gse.loc[df_gse['Sex'] == 'F',  f"{epi_est}Acc"].values
                    vals_m = df_gse.loc[df_gse['Sex'] == 'M',  f"{epi_est}Acc"].values
                    pval = df_stat_ss.at[epi_est, 'pval_fdr_bh']
                    age_vio_labels[epi_est] = f"{epi_est}<br>p-value: {pval:0.2e}"
                    if len(vals_f) > 0 and len(vals_m) > 0:
                        fig.add_trace(
                            go.Violin(
                                y=[epi_est_id] * len(vals_m),
                                x=vals_m,
                                name=age_vio_labels[epi_est],
                                box_visible=True,
                                box_width=0.8,
                                meanline_visible=True,
                                showlegend=False,
                                line_color='black',
                                fillcolor=colors_sex['M'],
                                marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                                points=False,
                                bandwidth=np.ptp(vals_m) / 25,
                                opacity=0.8,
                                legendgroup=age_vio_labels[epi_est],
                                scalegroup=age_vio_labels[epi_est],
                                side='negative',
                                orientation='h',
                                scalemode="width",
                                pointpos=-1.5,
                            ),
                            row=1,
                            col=1
                        )
                        fig.add_trace(
                            go.Violin(
                                y=[epi_est_id] * len(vals_f),
                                x=vals_f,
                                name=age_vio_labels[epi_est],
                                box_visible=True,
                                box_width=0.8,
                                meanline_visible=True,
                                showlegend=False,
                                line_color='black',
                                fillcolor=colors_sex['F'],
                                marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                                points=False,
                                bandwidth=np.ptp(vals_f) / 25,
                                opacity=0.8,
                                legendgroup=age_vio_labels[epi_est],
                                scalegroup=age_vio_labels[epi_est],
                                scalemode="width",
                                side='positive',
                                orientation='h',
                                pointpos=1.5,
                            ),
                            row=1,
                            col=1
                        )
                fig.update_layout(template="none")
                fig.update_xaxes(
                    title_text="Epigenetic Age Acceleration",
                    tickfont=dict(size=26),
                    titlefont=dict(size=26),
                    showgrid=True,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=1,
                    col=1
                )
                fig.update_yaxes(
                    tickmode='array',
                    tickvals=list(range(len(ages_ordered))),
                    ticktext=[color_tick('black', age_vio_labels[x]) for x in ages_ordered],
                    tickfont=dict(size=25),
                    showgrid=False,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=1,
                    col=1
                )
                fig.update_yaxes(autorange=False, range=[-0.5, len(ages) - 0.5], row=1, col=1)
                fig.update_xaxes(autorange=True, row=1, col=1)
            
                vals_f = df_gse.loc[df_gse['Sex'] == 'F', pace].values
                vals_m = df_gse.loc[df_gse['Sex'] == 'M', pace].values
                pval = df_stat_ss.at[pace, 'pval_fdr_bh']
                age_vio_labels[pace] = f"{pace}<br>p-value: {pval:0.2e}"
                if len(vals_f) > 0 and len(vals_m) > 0:
                    fig.add_trace(
                        go.Violin(
                            y=[0] * len(vals_m),
                            x=vals_m,
                            name=age_vio_labels[pace],
                            box_visible=True,
                            box_width=0.8,
                            meanline_visible=True,
                            showlegend=False,
                            line_color='black',
                            fillcolor=colors_sex['M'],
                            marker=dict(color=colors_sex['M'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                            points=False,
                            bandwidth=np.ptp(vals_m) / 25,
                            opacity=0.8,
                            legendgroup=age_vio_labels[pace],
                            scalegroup=age_vio_labels[pace],
                            side='negative',
                            orientation='h',
                            scalemode="width",
                            pointpos=-1.5,
                        ),
                        row=2,
                        col=1
                    )
                    fig.add_trace(
                        go.Violin(
                            y=[0] * len(vals_f),
                            x=vals_f,
                            name=age_vio_labels[pace],
                            box_visible=True,
                            box_width=0.8,
                            meanline_visible=True,
                            showlegend=False,
                            line_color='black',
                            fillcolor=colors_sex['F'],
                            marker=dict(color=colors_sex['F'], line=dict(color='black', width=0.35), opacity=0.8, size=8),
                            points=False,
                            bandwidth=np.ptp(vals_f) / 25,
                            opacity=0.8,
                            legendgroup=age_vio_labels[pace],
                            scalegroup=age_vio_labels[pace],
                            scalemode="width",
                            side='positive',
                            orientation='h',
                            pointpos=1.5,
                        ),
                        row=2,
                        col=1
                    )
                fig.update_xaxes(
                    title_text="Pace of Aging",
                    tickfont=dict(size=26),
                    titlefont=dict(size=26),
                    showgrid=True,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=2,
                    col=1
                )
                fig.update_yaxes(
                    tickmode='array',
                    tickvals=[0],
                    ticktext=[color_tick('black', age_vio_labels[pace])],
                    tickfont=dict(size=25),
                    showgrid=False,
                    autorange=True,
                    zeroline=False,
                    linecolor='black',
                    showline=True,
                    gridcolor='gainsboro',
                    gridwidth=0.001,
                    mirror="allticks",
                    ticks='outside',
                    showticklabels=True,
                    tickangle=0,
                    exponentformat='e',
                    showexponent='all',
                    row=2,
                    col=1
                )
                fig.update_yaxes(autorange=False, range=[-0.5, 0.5], row=2, col=1)
                fig.update_xaxes(autorange=True, row=2, col=1)
                
                fig.update_layout(
                    title=dict(xref='paper', x=1.0),
                    legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.01,
                        xanchor="left",
                        x=0.0001,
                        itemsizing='constant',
                        font_size=22
                    ),
                )
                fig.update_layout(
                    violingap=0.05,
                    violingroupgap=0.05,
                    height=140 * (len(ages) + 1),
                    width=1050,
                    margin=go.layout.Margin(
                        l=300,
                        r=30,
                        b=110,
                        t=30,
                        pad=0,
                    )
                )
                fig.write_image(f"{path_save}/violins.png", scale=2)
                fig.write_image(f"{path_save}/violins.pdf", format="pdf")
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_gse"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)    
            df_ss.to_excel(f"{path_save}/ss.xlsx", index_label='GSE')
            df_ss['GSE'] = df_ss.index.values
            df_ss_melt = pd.melt(df_ss, id_vars=['GSE'], value_vars=['F older M', 'M older F', 'Insignificant / One sex'], var_name='EAA type', value_name='Number of EAA estimators')
            fig, ax = plt.subplots(figsize=(3, 1.5 + 0.8 * len(gses)))
            sns.set_theme(style='whitegrid')
            barplot = sns.barplot(
                data=df_ss_melt,
                y='GSE',
                hue='EAA type',
                x='Number of EAA estimators',
                edgecolor='black',
                palette={'F older M': 'red', 'M older F': 'blue', 'Insignificant / One sex': 'gray'},
                ax=ax
            )
            ax.set_xlabel('Number of EAA estimators', fontsize=20)
            ax.set_ylabel('', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            for container in ax.containers:
                ax.bar_label(container, fmt='%d', fontsize=12)
            sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1))
            plt.savefig(f"{path_save}/ss_barplot.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/ss_barplot.pdf", bbox_inches='tight')
            plt.close(fig)
            
            n_rows = 2
            n_cols = 6
            fig_width = 15
            fig_height = 8
            low_percent = 0.005
            hgh_percent = 0.995
            ptp_shift = 0.1
    
            for gse in gses:
                df_gse = df_state.loc[df_state['GSE'] == gse,  :]

                x_min = df_gse['Age'].min()
                x_max = df_gse['Age'].max()
                x_ptp = x_max - x_min
    
                y_min = df_gse[list(ages)].min().min()
                y_max = df_gse[list(ages)].max().max()
                y_ptp = y_max - y_min
    
                y_percentiles = df_gse[ages].quantile([low_percent, hgh_percent])
                y_low = np.min(y_percentiles.loc[low_percent, :].values)
                y_hgh = np.max(y_percentiles.loc[hgh_percent, :].values)
                
                path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_gse/{gse}/sex_specific"
                pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
                
                fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
                sns.set_theme(style='whitegrid')
                
                for epi_est, params in ages_kde.items():
                    if epi_est == 'PC-GrimAge2':
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        axs[params['row_id'], params['col_id']].axis('off')
                    else:
                        for sex in ['M', 'F']:
                            try:
                                kdeplot = sns.kdeplot(
                                    data=df_gse.loc[df_gse['Sex'] == sex, :],
                                    x='Age',
                                    y=epi_est,
                                    fill=True,
                                    cbar=False,
                                    alpha=0.6,
                                    cut=0,
                                    color=colors_sex[sex],
                                    # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                                    legend=False,
                                    ax=axs[params['row_id'], params['col_id']]
                                )
                            except linalg.LinAlgError:
                                print(f"LinAlgError: {tissue} {state} {gse}")
                            
                        points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
                        axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)
                        
                        axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
                        axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
                        axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        if params['x_label'] == '':
                            axs[params['row_id'], params['col_id']].set_xticklabels([])
                
                fig.tight_layout()    
                plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
                plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
                plt.close(fig)
                
                path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/by_gse/{gse}"
                pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
                
                fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=True, gridspec_kw={'hspace': 0.075})
                sns.set_theme(style='whitegrid')
    
                for epi_est, params in ages_kde.items():
                    if epi_est == 'PC-GrimAge2':
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        axs[params['row_id'], params['col_id']].axis('off')
                    else:
                        kdeplot = sns.kdeplot(
                            data=df_gse,
                            x='Age',
                            y=epi_est,
                            fill=True,
                            cbar=False,
                            color=colors_gses[gse],
                            cut=0,
                            # clip=((x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp), (y_min - ptp_shift * y_ptp, y_max + ptp_shift * y_ptp)),
                            legend=False,
                            ax=axs[params['row_id'], params['col_id']]
                        )
                        regplot = sns.regplot(
                            data=df_gse,
                            x='Age',
                            y=epi_est,
                            scatter=False,
                            color='black',
                            truncate=True,
                            ax=axs[params['row_id'], params['col_id']]
                        )
                        points_unity = [min(x_min - ptp_shift * x_ptp, y_min - ptp_shift * y_ptp), max(x_max + ptp_shift * x_ptp, y_max + ptp_shift * y_ptp)]
                        axs[params['row_id'], params['col_id']].plot(points_unity, points_unity, color='black', marker=None, linestyle='--', linewidth=1.0)
                        
                        if df_gse.shape[0] > 1:
                            corr, _ = stats.pearsonr(df_gse['Age'].values, df_gse[epi_est].values)
                            mae = mean_absolute_error(df_gse['Age'].values, df_gse[epi_est].values)
                            label = r'$\rho$ = ' + f"{corr:0.2f}"
                            axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.10), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)
                            label = f"MAE = {mae:0.2f}"
                            axs[params['row_id'], params['col_id']].annotate(label, xy = (0.5, 0.02), size=16, xycoords=axs[params['row_id'], params['col_id']].transAxes, ha='center', color='black', alpha=0.75)
    
                        axs[params['row_id'], params['col_id']].set_xlim([x_min - ptp_shift * x_ptp, x_max + ptp_shift * x_ptp])
                        axs[params['row_id'], params['col_id']].set_ylim([y_low, y_hgh])
                        axs[params['row_id'], params['col_id']].set_title(params['title'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_ylabel(params['y_label'], fontsize=18)
                        axs[params['row_id'], params['col_id']].set_xlabel(params['x_label'])
                        if params['x_label'] == '':
                            axs[params['row_id'], params['col_id']].set_xticklabels([])
    
                fig.tight_layout()    
                plt.savefig(f"{path_save}/kde_ages.png", bbox_inches='tight', dpi=200)
                plt.savefig(f"{path_save}/kde_ages.pdf", bbox_inches='tight')
                plt.close(fig)
            
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/gses"
            pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)
            
            hist_bins = np.linspace(0, 110, 12)
            fig, ax = plt.subplots()
            sns.set_theme(style='whitegrid')
            histplot = sns.histplot(
                data=df_state,
                x=f"Age",
                bins=hist_bins,
                edgecolor='k',
                linewidth=1,
                hue_order=list(colors_gses.keys())[::-1],
                hue="GSE",
                palette=colors_gses,
                multiple="stack",
                ax=ax
            )
            plt.savefig(f"{path_save}/histplot.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/histplot.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(3, 1.5 + 0.8 * len(gses)))
            sns.set_theme(style='whitegrid')
            barplot = sns.barplot(
                data=dfs_states_gse_by_tissue[tissue][state],
                y=dfs_states_gse_by_tissue[tissue][state].index.values,
                hue=dfs_states_gse_by_tissue[tissue][state].index.values,
                x=dfs_states_gse_by_tissue[tissue][state]['GSE'].values,
                edgecolor='black',
                palette=colors_gses,
                dodge=False,
                ax=ax
            )
            ax.set_xlabel('Number of samples', fontsize=20)
            ax.set_ylabel('', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize=14)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.get_legend().remove()
            for container in ax.containers:
                ax.bar_label(container, fmt='%d', fontsize=18)
            plt.savefig(f"{path_save}/barplot.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/barplot.pdf", bbox_inches='tight')
            plt.close(fig)
            
            df_gses_aerr_mean = pd.DataFrame(index=gses, columns=ages, data=np.zeros(shape=(len(gses), len(ages))))
            df_gses_pace_mean = pd.DataFrame(index=gses, columns=[pace], data=np.zeros(shape=(len(gses), 1)))
            for gse in gses:
                vals = df_state.loc[df_state['GSE'] == gse, pace].values
                df_gses_pace_mean.at[gse, pace] = np.mean(vals)
                for age_type in ages:
                    vals = df_state.loc[df_state['GSE'] == gse, f"{age_type}Acc"].values
                    df_gses_aerr_mean.at[gse, age_type] = np.mean(vals)
            df_gses_aerr_mean.to_excel(f"{path_save}/aerr_mean.xlsx", index_label="GSE")
            df_gses_pace_mean.to_excel(f"{path_save}/pace_mean.xlsx", index_label="GSE")
            
            fig, ax = plt.subplots(figsize=(2.7 + 0.375 * len(ages), 1.8 + 0.15 * len(gses)))
            sns.set_theme(style='whitegrid')
            heatmap = sns.heatmap(
                df_gses_aerr_mean,
                annot=True,
                fmt=".1f",
                center=0.0,
                cmap='seismic',
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 35 / np.sqrt(max(df_gses_aerr_mean.shape))},
                ax=ax,
            )
            ax.set_xlabel('Epigenetic Age', fontsize=16)
            ax.set_ylabel('GSE', fontsize=16)
            ax.figure.axes[-1].set_ylabel(r"$\langle$ Acceleration $\rangle$", size=16)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            heatmap_order_ages = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/heatmap_aerr_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/heatmap_aerr_mean.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(gses)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='Age',
                y='GSE',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=heatmap_order_ages,
                palette=colors_gses,
                ax=ax
            )
            ax.set_xlabel('Chronological Age', fontsize=20)
            ax.set_ylabel('GSEs', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_age_for_heatmap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_age_for_heatmap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            if len(gses) > 2:
                figsize_shift_x = 0.35
                row_cluster = True
            else:
                figsize_shift_x = 0.125
                row_cluster = False
            sns.set_theme(style='whitegrid')
            clustermap = sns.clustermap(
                df_gses_aerr_mean,
                annot=True,
                col_cluster=True,
                row_cluster=row_cluster,
                fmt=".1f",
                center=0.0,
                cmap='seismic',
                linewidth=0.1,
                linecolor='black',
                tree_kws=dict(linewidths=1.5),
                annot_kws={"size": 55 / np.sqrt(max(df_gses_aerr_mean.shape))},
                figsize=((figsize_shift_x + 0.065 * len(ages)) * 10, (0.45 + 0.035 * len(gses)) * 10)
            )
            clustermap.ax_heatmap.set_xlabel('Epigenetic Age', fontsize=20)
            clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xmajorticklabels(), fontsize=18)
            clustermap.ax_heatmap.set_ylabel('GSE', fontsize=20)
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize=18, rotation=0)
            clustermap.ax_cbar.set_ylabel(r"$\langle$ Acceleration $\rangle$", size=20)
            clustermap.ax_cbar.tick_params(labelsize=18)
            clustermap_order_ages = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
            for spine in clustermap.ax_cbar.spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in clustermap.ax_heatmap.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/clustermap_aerr_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/clustermap_aerr_mean.pdf", bbox_inches='tight')
            plt.close(clustermap.fig)
            
            fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(gses)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='Age',
                y='GSE',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=clustermap_order_ages,
                palette=colors_gses,
                ax=ax
            )
            ax.set_xlabel('Chronological Age', fontsize=20)
            ax.set_ylabel('GSEs', fontsize=20)
            ax.set_xticklabels([f"{int(tick):d}" for tick in ax.get_xticks()], fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_age_for_clustermap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_age_for_clustermap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(1.5, 1.8 + 0.15 * len(gses)))
            sns.set_theme(style='whitegrid')
            heatmap = sns.heatmap(
                df_gses_pace_mean,
                annot=True,
                fmt=".3f",
                center=1.0,
                cmap='PiYG_r',
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 35 / np.sqrt(max(df_gses_aerr_mean.shape))},
                ax=ax
            )
            ax.set_xticklabels([''])
            ax.set_xlabel('DunedinPACE', fontsize=16)
            ax.set_ylabel('GSE', fontsize=16)
            ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$", size=16)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            heatmap_order_pace = [tick_label.get_text() for tick_label in ax.get_yticklabels()]
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/heatmap_pace_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/heatmap_pace_mean.pdf", bbox_inches='tight')
            plt.close(fig)
            
            fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(gses)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='DunedinPACE',
                y='GSE',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=heatmap_order_pace,
                palette=colors_gses,
                ax=ax
            )
            ax.set_xlabel('DunedinPace', fontsize=20)
            ax.set_ylabel('GSEs', fontsize=20)
            ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_pace_for_heatmap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_pace_for_heatmap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            if len(gses) > 2:
                figsize_shift_y = 0.35
                row_cluster = True
            else:
                figsize_shift_y = 0.25
                row_cluster = False
            sns.set_theme(style='whitegrid')
            clustermap = sns.clustermap(
                df_gses_pace_mean,
                annot=True,
                col_cluster=False,
                row_cluster=row_cluster,
                fmt=".3f",
                center=1.0,
                cmap='PiYG_r',
                linewidth=0.1,
                linecolor='black',
                tree_kws=dict(linewidths=1.5),
                dendrogram_ratio=(0.6, 0.0),
                cbar_pos=(0.15, 1.06, 0.9, 0.04),
                cbar_kws={"orientation": "horizontal"},
                annot_kws={"size": 55 / np.sqrt(max(df_gses_aerr_mean.shape))},
                figsize=(4, (figsize_shift_y + 0.03 * len(gses)) * 10)
            )
            clustermap.ax_heatmap.set_xlabel('DunedinPACE', fontsize=20)
            clustermap.ax_heatmap.set_xticklabels("", fontsize = 18)
            clustermap.ax_heatmap.set_ylabel('GSE', fontsize=20)
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), fontsize=18, rotation=0)
            clustermap.ax_cbar.set_title(r"$\langle$ Pace of Aging $\rangle$", size=20)
            clustermap.ax_cbar.tick_params(labelsize=18)
            for spine in clustermap.ax_cbar.spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_ymajorticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            clustermap_order_pace = [tick_label.get_text() for tick_label in clustermap.ax_heatmap.get_ymajorticklabels()]
            for tick_label in clustermap.ax_heatmap.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/clustermap_pace_mean.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/clustermap_pace_mean.pdf", bbox_inches='tight')
            plt.close(clustermap.fig)
            
            fig, ax = plt.subplots(figsize=(6, 1.5 + 0.8 * len(gses)))
            sns.set_theme(style='whitegrid')
            violin = sns.violinplot(
                data=df_state,
                x='DunedinPACE',
                y='GSE',
                orient='h',
                inner='box',
                cut=0,
                scale='width',
                order=clustermap_order_pace,
                palette=colors_gses,
                ax=ax
            )
            ax.set_xlabel('DunedinPace', fontsize=20)
            ax.set_ylabel('GSEs', fontsize=20)
            ax.set_xticklabels([f"{tick:.2f}" for tick in ax.get_xticks()], fontsize = 16)
            ax.set_yticklabels(ax.get_yticklabels(), fontsize = 18)
            ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
            plt.savefig(f"{path_save}/violin_pace_for_clustermap.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/violin_pace_for_clustermap.pdf", bbox_inches='tight')
            plt.close(fig)
            
            path_save = f"{path_load}/figures/epi_est_stat/by_tissue/{tissue}/by_states/{state}/gses/epi_ests"
            pathlib.Path(f"{path_save}/epi_ests").mkdir(parents=True, exist_ok=True)
            dfs_gses_gses_stat = {}
            for epi_est in ages + [pace]:
                dfs_gses_gses_stat[epi_est] = pd.DataFrame(index=gses, columns=gses, data=np.zeros(shape=(len(gses), len(gses))))
                if epi_est == pace:
                    col = epi_est
                else:
                    col = f"{epi_est}Acc"
                for gse_1_id, gse_1 in enumerate(gses):
                    vals_1 = df_state.loc[df_state['GSE'] == gse_1,  col].values
                    for gse_2_id in range(gse_1_id, len(gses)):
                        gse_2 = gses[gse_2_id]
                        vals_2 = df_state.loc[df_state['GSE'] == gse_2, col].values
                        if gse_1 != gse_2:
                            _, pval = mannwhitneyu(vals_1, vals_2, alternative='two-sided')
                            diff = np.mean(vals_2) - np.mean(vals_1)
                            dfs_gses_gses_stat[epi_est].at[gse_1, gse_2] = diff
                            dfs_gses_gses_stat[epi_est].at[gse_2, gse_1] = pval
                        else:
                            dfs_gses_gses_stat[epi_est].at[gse_1, gse_2]  = np.nan
                selection = np.tri(len(gses), len(gses), -1, dtype=np.bool)
                df_fdr = dfs_gses_gses_stat[epi_est].where(selection).stack().reset_index()
                df_fdr.columns = ['row', 'col', 'pval']
                _, df_fdr['pval_fdr_bh'], _, _ = multipletests(df_fdr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
                nzmin = df_fdr['pval_fdr_bh'][df_fdr['pval_fdr_bh'].gt(0)].min(0) * 0.5
                df_fdr['pval_fdr_bh'].replace({0.0: nzmin}, inplace=True)
                df_fdr['pval_fdr_bh_log'] = -np.log10(df_fdr.loc[:, 'pval_fdr_bh'].values)
                for line_id in range(df_fdr.shape[0]):
                    dfs_gses_gses_stat[epi_est].loc[df_fdr.at[line_id, 'row'], df_fdr.at[line_id, 'col']] = df_fdr.at[line_id, 'pval_fdr_bh_log']
                dfs_gses_gses_stat[epi_est].to_excel(f"{path_save}/{epi_est}_stat.xlsx", index_label="gseName")
                
                fig, ax = plt.subplots(figsize=(4.8 + 0.2 * len(gses), 0.8 + 0.2 * len(gses)))
                sns.set_theme(style='whitegrid')
                cmap_triu = plt.get_cmap("seismic").copy()
                heatmap_diff = sns.heatmap(
                    dfs_gses_gses_stat[epi_est],
                    mask=np.tri(len(gses), len(gses), -1, dtype=np.bool),
                    annot=True,
                    fmt=".2f",
                    center=0.0,
                    cmap=cmap_triu,
                    linewidth=0.1,
                    linecolor='black',
                    annot_kws={"size": 25 / np.sqrt(len(gses))},
                    ax=ax
                )
                if epi_est == pace:
                    ax.figure.axes[-1].set_ylabel(r"$\langle$ Pace of Aging $\rangle$ Difference", size=13)
                else:
                    ax.figure.axes[-1].set_ylabel(r"$\langle$ Age Acceleration $\rangle$ Difference", size=13)
                for spine in ax.figure.axes[-1].spines.values():
                    spine.set(visible=True, lw=0.25, edgecolor="black")
                cmap_tril = plt.get_cmap("cool").copy()
                cmap_tril.set_under('black')
                heatmap_pval = sns.heatmap(
                    dfs_gses_gses_stat[epi_est],
                    mask=np.tri(len(gses), len(gses), -1, dtype=np.bool).T,
                    annot=True,
                    fmt=".1f",
                    vmin=-np.log10(0.05),
                    cmap=cmap_tril,
                    linewidth=0.1,
                    linecolor='black',
                    annot_kws={"size": 25 / np.sqrt(len(gses))},
                    ax=ax
                )
                ax.figure.axes[-1].set_ylabel(r"$-\log_{10}(\mathrm{p-value})$", size=13)
                for spine in ax.figure.axes[-1].spines.values():
                    spine.set(visible=True, lw=0.25, edgecolor="black")
                ax.set_xlabel('', fontsize=16)
                ax.set_ylabel('', fontsize=16)
                ax.set_title(epi_est, fontsize=16)
                ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
                for tick_label in ax.get_xticklabels():
                    tick_label.set_color(colors_gses[tick_label.get_text()])
                    ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
                for tick_label in ax.get_yticklabels():
                    tick_label.set_color(colors_gses[tick_label.get_text()])
                    ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
                plt.savefig(f"{path_save}/{epi_est}_stat.png", bbox_inches='tight', dpi=200)
                plt.savefig(f"{path_save}/{epi_est}_stat.pdf", bbox_inches='tight')
                plt.close(fig)
            
            df_gses_gses_stat = pd.DataFrame(index=gses, columns=gses, data=np.zeros(shape=(len(gses), len(gses))))
            for gse_1_id, gse_1 in enumerate(gses):
                for gse_2_id in range(gse_1_id, len(gses)):
                    gse_2 = gses[gse_2_id]
                    for epi_est in ages + [pace]:
                        pval = dfs_gses_gses_stat[epi_est].at[gse_2, gse_1]
                        diff = dfs_gses_gses_stat[epi_est].at[gse_1, gse_2]
                        if diff > 0:
                            df_gses_gses_stat.at[gse_1, gse_2] += 1
                        else:
                            df_gses_gses_stat.at[gse_1, gse_2] -= 1
                        df_gses_gses_stat.at[gse_2, gse_1] += pval / (len(ages + [pace]))
            df_gses_gses_stat.to_excel(f"{path_save}/global_stat.xlsx", index_label="GSE")
            fig, ax = plt.subplots(figsize=(4.8 + 0.2 * len(gses), 0.8 + 0.2 * len(gses)))
            sns.set_theme(style='whitegrid')
            cmap_triu = plt.get_cmap("seismic").copy()
            sns.heatmap(
                df_gses_gses_stat,
                mask=np.tri(len(gses), len(gses), -1, dtype=np.bool),
                annot=True,
                cmap=cmap_triu,
                center=0.0,
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 25 / np.sqrt(max(df_gses_aerr_mean.shape))},
                ax=ax
            )
            ax.figure.axes[-1].set_ylabel(r"Aggregated Sign of $\langle$ EAA $\rangle$ Difference", size=13)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            cmap_tril = plt.get_cmap("cool").copy()
            cmap_tril.set_under('black')
            sns.heatmap(
                df_gses_gses_stat,
                mask=np.tri(len(gses), len(gses), -1, dtype=np.bool).T,
                annot=True,
                fmt=".1f",
                vmin=-np.log10(0.05),
                cmap=cmap_tril,
                linewidth=0.1,
                linecolor='black',
                annot_kws={"size": 25 / np.sqrt(max(df_gses_aerr_mean.shape))},
                ax=ax
            )
            ax.figure.axes[-1].set_ylabel(r"$\langle -\log_{10}(\mathrm{p-value}) \rangle$", size=13)
            for spine in ax.figure.axes[-1].spines.values():
                spine.set(visible=True, lw=0.25, edgecolor="black")
            ax.set_xlabel('', fontsize=16)
            ax.set_ylabel('', fontsize=16)
            ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_xticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
                ax.set_xticklabels(ax.get_xticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            for tick_label in ax.get_yticklabels():
                tick_label.set_color(colors_gses[tick_label.get_text()])
                ax.set_yticklabels(ax.get_yticklabels(), path_effects=[pe.withStroke(linewidth=0.2, foreground="black")])
            plt.savefig(f"{path_save}/global_stat.png", bbox_inches='tight', dpi=200)
            plt.savefig(f"{path_save}/global_stat.pdf", bbox_inches='tight')
            plt.close(fig)